In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
df_data_general = pd.read_csv('../../../data/data_general.csv')

df_data_general['INITIAL_TIME'] = pd.to_datetime(df_data_general['INITIAL_TIME'])
df_data_general['FINAL_TIME'] = pd.to_datetime(df_data_general['FINAL_TIME'])

df_data_general['INITIAL_TIME'] = df_data_general['INITIAL_TIME'].dt.to_period('D')
df_data_general['INITIAL_TIME'] = df_data_general['INITIAL_TIME'].dt.to_timestamp()


df_data_general['FINAL_TIME'] = df_data_general['FINAL_TIME'].dt.to_period('D')
df_data_general['FINAL_TIME'] = df_data_general['FINAL_TIME'].dt.to_timestamp()

df_data_general['Weekday']= df_data_general['INITIAL_TIME'].dt.strftime('%A')
df_data_general['number_of_day'] = df_data_general['INITIAL_TIME'].dt.day_of_week

df_data_general['TIME_ON_DEVICE_MIN'] = df_data_general['TIME_ON_DEVICE_SEC'] / 60

df_data_general['Hour'] = df_data_general['INITIAL_TIME'].dt.hour
df_data_general['Weekday'] = df_data_general['INITIAL_TIME'].dt.weekday   # 0=Lunes, 6=Domingo
df_data_general['Weekend'] = (df_data_general['Weekday'] >= 5).astype(int)
df_data_general['Month'] = df_data_general['INITIAL_TIME'].dt.month

df_data_general = df_data_general[df_data_general['TIME_ON_DEVICE_MIN'] < 600 ]

df_data_general = df_data_general[df_data_general['WIN_TOTAL'] > 0]
df_data_general['NET_SPEND'] = df_data_general['FINAL_AMOUNT'] - df_data_general['INITIAL_AMOUNT']
df_data_general = df_data_general[df_data_general['NET_SPEND'] < 10000 ]

In [ ]:
# import numpy as np
# import pandas as pd
# import os
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import RobustScaler, MinMaxScaler
# from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
# from sklearn.linear_model import LinearRegression
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# import warnings
# warnings.filterwarnings('ignore')

# class MultiArchitectureCasinoML:
#     def __init__(self):
#         self.scalers = {}
#         self.models = {}
#         self.feature_names = {}
#         self.model_types = ['MLP', 'RNN', 'LSTM', 'GRU', 'CNN1D', 'CNN2D', 'TRANSFORMER', 'AUTOENCODER']
        
#     def filter_outliers(self, df, method='iqr', factor=1.5):
#         """Filtrar outliers del dataset"""
#         print(f"\n=== FILTRADO DE OUTLIERS (Método: {method.upper()}) ===")
#         print(f"Dataset original: {len(df)} registros")
        
#         df_filtered = df.copy()
#         numeric_cols = ['BET_TOTAL', 'WIN_TOTAL', 'TIME_ON_DEVICE_MIN', 'INITIAL_AMOUNT']
        
#         if method == 'iqr':
#             for col in numeric_cols:
#                 if col in df_filtered.columns:
#                     Q1 = df_filtered[col].quantile(0.25)
#                     Q3 = df_filtered[col].quantile(0.75)
#                     IQR = Q3 - Q1
#                     lower_bound = Q1 - factor * IQR
#                     upper_bound = Q3 + factor * IQR
                    
#                     before_count = len(df_filtered)
#                     df_filtered = df_filtered[
#                         (df_filtered[col] >= lower_bound) & 
#                         (df_filtered[col] <= upper_bound)
#                     ]
#                     removed = before_count - len(df_filtered)
#                     if removed > 0:
#                         print(f"{col}: Removidos {removed} outliers (rango: {lower_bound:.1f} - {upper_bound:.1f})")
        
#         elif method == 'zscore':
#             from scipy import stats
#             for col in numeric_cols:
#                 if col in df_filtered.columns:
#                     z_scores = np.abs(stats.zscore(df_filtered[col]))
#                     before_count = len(df_filtered)
#                     df_filtered = df_filtered[z_scores < 3]
#                     removed = before_count - len(df_filtered)
#                     if removed > 0:
#                         print(f"{col}: Removidos {removed} outliers (z-score >= 3)")
        
#         elif method == 'percentile':
#             for col in numeric_cols:
#                 if col in df_filtered.columns:
#                     lower_percentile = df_filtered[col].quantile(0.02)
#                     upper_percentile = df_filtered[col].quantile(0.98)
#                     before_count = len(df_filtered)
#                     df_filtered = df_filtered[
#                         (df_filtered[col] >= lower_percentile) & 
#                         (df_filtered[col] <= upper_percentile)
#                     ]
#                     removed = before_count - len(df_filtered)
#                     if removed > 0:
#                         print(f"{col}: Removidos {removed} outliers (percentiles 2-98)")
        
#         print(f"Dataset final: {len(df_filtered)} registros")
#         print(f"Total removidos: {len(df) - len(df_filtered)} ({(len(df) - len(df_filtered))/len(df)*100:.1f}%)")
        
#         return df_filtered
    
#     def create_business_features(self, df, tiempo_pred=None, bet_pred=None):
#         """Crear features que reflejen la lógica real del negocio de casino"""
#         features = df[['INITIAL_AMOUNT', 'AVG_BET', 'Cluster']].copy()
#         feature_names = ['INITIAL_AMOUNT', 'AVG_BET', 'Cluster']
        
#         if tiempo_pred is not None:
#             features['tiempo_pred'] = tiempo_pred
#             feature_names.append('tiempo_pred')
            
#         if bet_pred is not None:
#             features['bet_pred'] = bet_pred
#             features['total_money_handled'] = bet_pred
#             features['house_edge_effect'] = bet_pred * 0.05
#             features['money_multiplier'] = bet_pred / (df['INITIAL_AMOUNT'] + 1)
#             features['reinvestment_indicator'] = np.where(bet_pred > df['INITIAL_AMOUNT'], 1, 0)
#             features['excess_betting'] = np.maximum(0, bet_pred - df['INITIAL_AMOUNT'])
#             features['money_at_risk'] = np.minimum(bet_pred, df['INITIAL_AMOUNT'])
#             features['cluster_risk_adjusted'] = df['Cluster'] * features['money_multiplier']
            
#             feature_names.extend(['bet_pred', 'total_money_handled', 'house_edge_effect', 
#                                 'money_multiplier', 'reinvestment_indicator', 
#                                 'excess_betting', 'money_at_risk', 'cluster_risk_adjusted'])
            
#         return features, feature_names
    
#     def prepare_sequence_data(self, X, sequence_length=10):
#         """Preparar datos para modelos secuenciales"""
#         if len(X) < sequence_length:
#             X_padded = np.pad(X, ((sequence_length - len(X), 0), (0, 0)), mode='edge')
#             X = X_padded
        
#         X_seq = []
#         for i in range(len(X) - sequence_length + 1):
#             X_seq.append(X[i:(i + sequence_length)])
        
#         if len(X_seq) == 0:
#             if len(X) == 0:
#                 return np.zeros((1, sequence_length, X.shape[1] if len(X.shape) > 1 else 1))
#             X_seq = [np.pad(X, ((max(0, sequence_length - len(X)), 0), (0, 0)), mode='edge')[-sequence_length:]]
            
#         return np.array(X_seq)
    
#     def prepare_cnn2d_data(self, X, height=4, width=None):
#         """Preparar datos para CNN2D"""
#         n_features = X.shape[1]
        
#         if width is None:
#             width = n_features // height
#             if n_features % height != 0:
#                 pad_size = height - (n_features % height)
#                 X = np.pad(X, ((0, 0), (0, pad_size)), mode='constant')
#                 n_features = X.shape[1]
#                 width = n_features // height
        
#         X_2d = X.reshape(X.shape[0], height, width, 1)
#         return X_2d
    
#     def create_mlp_model(self, input_shape, model_type='tiempo'):
#         """Crear modelo MLP"""
#         model = keras.Sequential()
        
#         if model_type == 'tiempo':
#             model.add(layers.Dense(128, activation='relu', input_shape=(input_shape,)))
#             model.add(layers.BatchNormalization())
#             model.add(layers.Dropout(0.3))
#             model.add(layers.Dense(64, activation='relu'))
#             model.add(layers.BatchNormalization())
#             model.add(layers.Dropout(0.2))
#             model.add(layers.Dense(32, activation='relu'))
#             model.add(layers.Dropout(0.1))
#             model.add(layers.Dense(1, activation='linear'))
            
#         elif model_type == 'bet':
#             model.add(layers.Dense(256, activation='relu', input_shape=(input_shape,)))
#             model.add(layers.BatchNormalization())
#             model.add(layers.Dropout(0.4))
#             model.add(layers.Dense(128, activation='relu'))
#             model.add(layers.BatchNormalization())
#             model.add(layers.Dropout(0.3))
#             model.add(layers.Dense(64, activation='relu'))
#             model.add(layers.BatchNormalization())
#             model.add(layers.Dropout(0.2))
#             model.add(layers.Dense(32, activation='relu'))
#             model.add(layers.Dropout(0.1))
#             model.add(layers.Dense(1, activation='linear'))
            
#         elif model_type == 'win':
#             model.add(layers.Dense(512, activation='relu', input_shape=(input_shape,)))
#             model.add(layers.BatchNormalization())
#             model.add(layers.Dropout(0.4))
#             model.add(layers.Dense(256, activation='relu'))
#             model.add(layers.BatchNormalization())
#             model.add(layers.Dropout(0.3))
#             model.add(layers.Dense(128, activation='relu'))
#             model.add(layers.BatchNormalization())
#             model.add(layers.Dropout(0.3))
#             model.add(layers.Dense(64, activation='relu'))
#             model.add(layers.BatchNormalization())
#             model.add(layers.Dropout(0.2))
#             model.add(layers.Dense(32, activation='relu'))
#             model.add(layers.Dropout(0.1))
#             model.add(layers.Dense(1, activation='linear'))
        
#         model.compile(
#             optimizer=keras.optimizers.Adam(learning_rate=0.001),
#             loss='mse',
#             metrics=['mae']
#         )
        
#         return model
    
#     def create_rnn_model(self, input_shape, model_type='tiempo'):
#         """Crear modelo RNN"""
#         model = keras.Sequential()
        
#         if model_type == 'tiempo':
#             model.add(layers.SimpleRNN(64, return_sequences=True, input_shape=input_shape))
#             model.add(layers.Dropout(0.3))
#             model.add(layers.SimpleRNN(32, return_sequences=False))
#             model.add(layers.Dropout(0.2))
#             model.add(layers.Dense(16, activation='relu'))
#             model.add(layers.Dense(1, activation='linear'))
            
#         elif model_type == 'bet':
#             model.add(layers.SimpleRNN(128, return_sequences=True, input_shape=input_shape))
#             model.add(layers.Dropout(0.4))
#             model.add(layers.SimpleRNN(64, return_sequences=True))
#             model.add(layers.Dropout(0.3))
#             model.add(layers.SimpleRNN(32, return_sequences=False))
#             model.add(layers.Dropout(0.2))
#             model.add(layers.Dense(16, activation='relu'))
#             model.add(layers.Dense(1, activation='linear'))
            
#         elif model_type == 'win':
#             model.add(layers.SimpleRNN(256, return_sequences=True, input_shape=input_shape))
#             model.add(layers.Dropout(0.4))
#             model.add(layers.SimpleRNN(128, return_sequences=True))
#             model.add(layers.Dropout(0.3))
#             model.add(layers.SimpleRNN(64, return_sequences=False))
#             model.add(layers.Dropout(0.2))
#             model.add(layers.Dense(32, activation='relu'))
#             model.add(layers.Dense(16, activation='relu'))
#             model.add(layers.Dense(1, activation='linear'))
        
#         model.compile(
#             optimizer=keras.optimizers.Adam(learning_rate=0.001),
#             loss='mse',
#             metrics=['mae']
#         )
        
#         return model
    
#     def create_lstm_model(self, input_shape, model_type='tiempo'):
#         """Crear modelo LSTM"""
#         model = keras.Sequential()
        
#         if model_type == 'tiempo':
#             model.add(layers.LSTM(64, return_sequences=True, input_shape=input_shape))
#             model.add(layers.Dropout(0.3))
#             model.add(layers.LSTM(32, return_sequences=False))
#             model.add(layers.Dropout(0.2))
#             model.add(layers.Dense(16, activation='relu'))
#             model.add(layers.Dense(1, activation='linear'))
            
#         elif model_type == 'bet':
#             model.add(layers.LSTM(128, return_sequences=True, input_shape=input_shape))
#             model.add(layers.Dropout(0.4))
#             model.add(layers.LSTM(64, return_sequences=True))
#             model.add(layers.Dropout(0.3))
#             model.add(layers.LSTM(32, return_sequences=False))
#             model.add(layers.Dropout(0.2))
#             model.add(layers.Dense(16, activation='relu'))
#             model.add(layers.Dense(1, activation='linear'))
            
#         elif model_type == 'win':
#             model.add(layers.LSTM(256, return_sequences=True, input_shape=input_shape))
#             model.add(layers.Dropout(0.4))
#             model.add(layers.LSTM(128, return_sequences=True))
#             model.add(layers.Dropout(0.3))
#             model.add(layers.LSTM(64, return_sequences=False))
#             model.add(layers.Dropout(0.2))
#             model.add(layers.Dense(32, activation='relu'))
#             model.add(layers.Dense(16, activation='relu'))
#             model.add(layers.Dense(1, activation='linear'))
        
#         model.compile(
#             optimizer=keras.optimizers.Adam(learning_rate=0.001),
#             loss='mse',
#             metrics=['mae']
#         )
        
#         return model
    
#     def create_gru_model(self, input_shape, model_type='tiempo'):
#         """Crear modelo GRU"""
#         model = keras.Sequential()
        
#         if model_type == 'tiempo':
#             model.add(layers.GRU(64, return_sequences=True, input_shape=input_shape))
#             model.add(layers.Dropout(0.3))
#             model.add(layers.GRU(32, return_sequences=False))
#             model.add(layers.Dropout(0.2))
#             model.add(layers.Dense(16, activation='relu'))
#             model.add(layers.Dense(1, activation='linear'))
            
#         elif model_type == 'bet':
#             model.add(layers.GRU(128, return_sequences=True, input_shape=input_shape))
#             model.add(layers.Dropout(0.4))
#             model.add(layers.GRU(64, return_sequences=True))
#             model.add(layers.Dropout(0.3))
#             model.add(layers.GRU(32, return_sequences=False))
#             model.add(layers.Dropout(0.2))
#             model.add(layers.Dense(16, activation='relu'))
#             model.add(layers.Dense(1, activation='linear'))
            
#         elif model_type == 'win':
#             model.add(layers.GRU(256, return_sequences=True, input_shape=input_shape))
#             model.add(layers.Dropout(0.4))
#             model.add(layers.GRU(128, return_sequences=True))
#             model.add(layers.Dropout(0.3))
#             model.add(layers.GRU(64, return_sequences=False))
#             model.add(layers.Dropout(0.2))
#             model.add(layers.Dense(32, activation='relu'))
#             model.add(layers.Dense(16, activation='relu'))
#             model.add(layers.Dense(1, activation='linear'))
        
#         model.compile(
#             optimizer=keras.optimizers.Adam(learning_rate=0.001),
#             loss='mse',
#             metrics=['mae']
#         )
        
#         return model
    
#     def create_cnn1d_model(self, input_shape, model_type='tiempo'):
#         """Crear modelo CNN 1D"""
#         model = keras.Sequential()
        
#         if model_type == 'tiempo':
#             model.add(layers.Conv1D(64, 3, activation='relu', input_shape=input_shape))
#             model.add(layers.BatchNormalization())
#             model.add(layers.MaxPooling1D(2))
#             model.add(layers.Conv1D(32, 3, activation='relu'))
#             model.add(layers.BatchNormalization())
#             model.add(layers.GlobalMaxPooling1D())
#             model.add(layers.Dense(50, activation='relu'))
#             model.add(layers.Dropout(0.3))
#             model.add(layers.Dense(1, activation='linear'))
            
#         elif model_type == 'bet':
#             model.add(layers.Conv1D(128, 3, activation='relu', input_shape=input_shape))
#             model.add(layers.BatchNormalization())
#             model.add(layers.MaxPooling1D(2))
#             model.add(layers.Conv1D(64, 3, activation='relu'))
#             model.add(layers.BatchNormalization())
#             model.add(layers.Conv1D(32, 3, activation='relu'))
#             model.add(layers.GlobalMaxPooling1D())
#             model.add(layers.Dense(100, activation='relu'))
#             model.add(layers.Dropout(0.4))
#             model.add(layers.Dense(50, activation='relu'))
#             model.add(layers.Dropout(0.3))
#             model.add(layers.Dense(1, activation='linear'))
            
#         elif model_type == 'win':
#             model.add(layers.Conv1D(256, 3, activation='relu', input_shape=input_shape))
#             model.add(layers.BatchNormalization())
#             model.add(layers.MaxPooling1D(2))
#             model.add(layers.Conv1D(128, 3, activation='relu'))
#             model.add(layers.BatchNormalization())
#             model.add(layers.Conv1D(64, 3, activation='relu'))
#             model.add(layers.BatchNormalization())
#             model.add(layers.GlobalMaxPooling1D())
#             model.add(layers.Dense(200, activation='relu'))
#             model.add(layers.Dropout(0.4))
#             model.add(layers.Dense(100, activation='relu'))
#             model.add(layers.Dropout(0.3))
#             model.add(layers.Dense(50, activation='relu'))
#             model.add(layers.Dropout(0.2))
#             model.add(layers.Dense(1, activation='linear'))
        
#         model.compile(
#             optimizer=keras.optimizers.Adam(learning_rate=0.001),
#             loss='mse',
#             metrics=['mae']
#         )
        
#         return model
    
#     def create_cnn2d_model(self, input_shape, model_type='tiempo'):
#         """Crear modelo CNN 2D"""
#         model = keras.Sequential()
        
#         if model_type == 'tiempo':
#             model.add(layers.Conv2D(32, (2, 2), activation='relu', input_shape=input_shape))
#             model.add(layers.BatchNormalization())
#             model.add(layers.Conv2D(16, (2, 2), activation='relu'))
#             model.add(layers.BatchNormalization())
#             model.add(layers.GlobalMaxPooling2D())
#             model.add(layers.Dense(32, activation='relu'))
#             model.add(layers.Dropout(0.3))
#             model.add(layers.Dense(1, activation='linear'))
            
#         elif model_type == 'bet':
#             model.add(layers.Conv2D(64, (2, 2), activation='relu', input_shape=input_shape))
#             model.add(layers.BatchNormalization())
#             model.add(layers.Conv2D(32, (2, 2), activation='relu'))
#             model.add(layers.BatchNormalization())
#             model.add(layers.Conv2D(16, (2, 2), activation='relu'))
#             model.add(layers.GlobalMaxPooling2D())
#             model.add(layers.Dense(64, activation='relu'))
#             model.add(layers.Dropout(0.4))
#             model.add(layers.Dense(32, activation='relu'))
#             model.add(layers.Dropout(0.3))
#             model.add(layers.Dense(1, activation='linear'))
            
#         elif model_type == 'win':
#             model.add(layers.Conv2D(128, (2, 2), activation='relu', input_shape=input_shape))
#             model.add(layers.BatchNormalization())
#             model.add(layers.Conv2D(64, (2, 2), activation='relu'))
#             model.add(layers.BatchNormalization())
#             model.add(layers.Conv2D(32, (2, 2), activation='relu'))
#             model.add(layers.BatchNormalization())
#             model.add(layers.GlobalMaxPooling2D())
#             model.add(layers.Dense(128, activation='relu'))
#             model.add(layers.Dropout(0.4))
#             model.add(layers.Dense(64, activation='relu'))
#             model.add(layers.Dropout(0.3))
#             model.add(layers.Dense(32, activation='relu'))
#             model.add(layers.Dropout(0.2))
#             model.add(layers.Dense(1, activation='linear'))
        
#         model.compile(
#             optimizer=keras.optimizers.Adam(learning_rate=0.001),
#             loss='mse',
#             metrics=['mae']
#         )
        
#         return model
    
#     def create_transformer_model(self, input_shape, model_type='tiempo'):
#         """Crear modelo Transformer"""
#         inputs = keras.Input(shape=input_shape)
        
#         if model_type == 'tiempo':
#             attention = layers.MultiHeadAttention(num_heads=4, key_dim=16)(inputs, inputs)
#             attention = layers.Dropout(0.3)(attention)
#             attention = layers.LayerNormalization()(attention + inputs)
            
#             ff = layers.Dense(64, activation='relu')(attention)
#             ff = layers.Dropout(0.3)(ff)
#             ff = layers.Dense(input_shape[-1])(ff)
#             ff = layers.LayerNormalization()(ff + attention)
            
#         elif model_type == 'win':
#             attention = layers.MultiHeadAttention(num_heads=8, key_dim=64)(inputs, inputs)
#             attention = layers.Dropout(0.4)(attention)
#             attention = layers.LayerNormalization()(attention + inputs)
            
#             ff = layers.Dense(256, activation='relu')(attention)
#             ff = layers.Dropout(0.4)(ff)
#             ff = layers.Dense(128, activation='relu')(ff)
#             ff = layers.Dropout(0.3)(ff)
#             ff = layers.Dense(64, activation='relu')(ff)
#             ff = layers.Dropout(0.2)(ff)
#             ff = layers.Dense(input_shape[-1])(ff)
#             ff = layers.LayerNormalization()(ff + attention)
        
#         pooled = layers.GlobalAveragePooling1D()(ff)
#         outputs = layers.Dense(1, activation='linear')(pooled)
        
#         model = keras.Model(inputs, outputs)
#         model.compile(
#             optimizer=keras.optimizers.Adam(learning_rate=0.001),
#             loss='mse',
#             metrics=['mae']
#         )
        
#         return model
    
#     def create_autoencoder_model(self, input_shape, model_type='tiempo'):
#         """Crear modelo Autoencoder"""
#         input_layer = keras.Input(shape=(input_shape,))
        
#         if model_type == 'tiempo':
#             encoded = layers.Dense(64, activation='relu')(input_layer)
#             encoded = layers.BatchNormalization()(encoded)
#             encoded = layers.Dropout(0.3)(encoded)
#             encoded = layers.Dense(32, activation='relu')(encoded)
#             encoded = layers.BatchNormalization()(encoded)
#             encoded = layers.Dropout(0.2)(encoded)
#             encoded = layers.Dense(16, activation='relu')(encoded)
            
#             decoded = layers.Dense(32, activation='relu')(encoded)
#             decoded = layers.BatchNormalization()(decoded)
#             decoded = layers.Dropout(0.2)(decoded)
#             decoded = layers.Dense(64, activation='relu')(decoded)
#             decoded = layers.BatchNormalization()(decoded)
#             decoded = layers.Dropout(0.3)(decoded)
#             decoded = layers.Dense(input_shape, activation='linear')(decoded)
            
#             predictor = layers.Dense(8, activation='relu')(encoded)
#             predictor = layers.Dropout(0.2)(predictor)
#             output = layers.Dense(1, activation='linear')(predictor)
            
#         elif model_type == 'bet':
#             encoded = layers.Dense(128, activation='relu')(input_layer)
#             encoded = layers.BatchNormalization()(encoded)
#             encoded = layers.Dropout(0.4)(encoded)
#             encoded = layers.Dense(64, activation='relu')(encoded)
#             encoded = layers.BatchNormalization()(encoded)
#             encoded = layers.Dropout(0.3)(encoded)
#             encoded = layers.Dense(32, activation='relu')(encoded)
            
#             decoded = layers.Dense(64, activation='relu')(encoded)
#             decoded = layers.BatchNormalization()(decoded)
#             decoded = layers.Dropout(0.3)(decoded)
#             decoded = layers.Dense(128, activation='relu')(decoded)
#             decoded = layers.BatchNormalization()(decoded)
#             decoded = layers.Dropout(0.4)(decoded)
#             decoded = layers.Dense(input_shape, activation='linear')(decoded)
            
#             predictor = layers.Dense(16, activation='relu')(encoded)
#             predictor = layers.Dropout(0.3)(predictor)
#             output = layers.Dense(1, activation='linear')(predictor)
            
#         elif model_type == 'win':
#             encoded = layers.Dense(256, activation='relu')(input_layer)
#             encoded = layers.BatchNormalization()(encoded)
#             encoded = layers.Dropout(0.4)(encoded)
#             encoded = layers.Dense(128, activation='relu')(encoded)
#             encoded = layers.BatchNormalization()(encoded)
#             encoded = layers.Dropout(0.3)(encoded)
#             encoded = layers.Dense(64, activation='relu')(encoded)
            
#             decoded = layers.Dense(128, activation='relu')(encoded)
#             decoded = layers.BatchNormalization()(decoded)
#             decoded = layers.Dropout(0.3)(decoded)
#             decoded = layers.Dense(256, activation='relu')(decoded)
#             decoded = layers.BatchNormalization()(decoded)
#             decoded = layers.Dropout(0.4)(decoded)
#             decoded = layers.Dense(input_shape, activation='linear')(decoded)
            
#             predictor = layers.Dense(32, activation='relu')(encoded)
#             predictor = layers.Dropout(0.3)(predictor)
#             predictor = layers.Dense(16, activation='relu')(predictor)
#             predictor = layers.Dropout(0.2)(predictor)
#             output = layers.Dense(1, activation='linear')(predictor)
        
#         model = keras.Model(input_layer, [output, decoded])
#         model.compile(
#             optimizer=keras.optimizers.Adam(learning_rate=0.001),
#             loss=['mse', 'mse'],
#             loss_weights=[0.7, 0.3],
#             metrics={'dense_output': ['mae'], 'decoded_output': ['mae']}
#         )
        
#         return model
    
#     def generate_random_data(self, X_train, n_samples=1000):
#         """Generar datos aleatorios basados en estadísticas de entrenamiento"""
#         np.random.seed(42)
#         random_data = np.zeros((n_samples, X_train.shape[1]))
        
#         for i in range(X_train.shape[1]):
#             feature_mean = X_train[:, i].mean()
#             feature_std = X_train[:, i].std()
#             feature_min = X_train[:, i].min()
#             feature_max = X_train[:, i].max()
            
#             random_data[:, i] = np.random.normal(feature_mean, feature_std, size=n_samples)
#             random_data[:, i] = np.clip(random_data[:, i], feature_min, feature_max)
        
#         return random_data
    
#     def create_individual_feature_visualizations(self, X_real_scaled, y_real, X_random_scaled, y_pred_random, 
#                                            X_real_original, X_random_original, feature_names, model_name, 
#                                            architecture, output_dir='visualizations', filter_outliers=True):
#         """Crear visualizaciones individuales para cada variable con valores sin escalar"""
#         if not os.path.exists(output_dir):
#             os.makedirs(output_dir)
        
#         n_features = X_real_original.shape[1]
#         print(f"\n=== CREANDO VISUALIZACIONES PARA {n_features} VARIABLES - {model_name.upper()} ({architecture}) ===")
        
#         # FIX: Ensure all arrays have the same length
#         min_len = min(len(X_real_original), len(y_real))
#         X_real_original = X_real_original[:min_len]
#         y_real = y_real[:min_len]
        
#         # Also ensure random data matches
#         min_random_len = min(len(X_random_original), len(y_pred_random))
#         X_random_original = X_random_original[:min_random_len]
#         y_pred_random = y_pred_random[:min_random_len]
        
#         print(f"Real data points: {len(X_real_original)}, Random data points: {len(X_random_original)}")
        
#         fig, axes = plt.subplots(2, (n_features + 1) // 2, figsize=(6 * ((n_features + 1) // 2), 12))
#         if n_features == 1:
#             axes = [axes]
#         elif (n_features + 1) // 2 == 1:
#             axes = axes.reshape(-1, 1)
#         axes = axes.flatten()
        
#         for i, feature_name in enumerate(feature_names):
#             if i >= len(axes):
#                 break
                
#             df_viz = pd.DataFrame({
#                 'Feature_Value': np.concatenate([X_real_original[:, i], X_random_original[:, i]]),
#                 'Predictions': np.concatenate([y_real, y_pred_random]),
#                 'Data_Type': ['Real'] * len(X_real_original) + ['Random'] * len(X_random_original)
#             })
            
#             if filter_outliers:
#                 feature_p5 = df_viz['Feature_Value'].quantile(0.05)
#                 feature_p95 = df_viz['Feature_Value'].quantile(0.95)
#                 pred_p5 = df_viz['Predictions'].quantile(0.05)
#                 pred_p95 = df_viz['Predictions'].quantile(0.95)
                
#                 mask = (
#                     (df_viz['Feature_Value'] >= feature_p5) & 
#                     (df_viz['Feature_Value'] <= feature_p95) &
#                     (df_viz['Predictions'] >= pred_p5) &
#                     (df_viz['Predictions'] <= pred_p95)
#                 )
                
#                 df_viz_filtered = df_viz[mask].copy()
#                 removed_points = len(df_viz) - len(df_viz_filtered)
                
#                 if removed_points > 0:
#                     print(f"{feature_name}: Removidos {removed_points} outliers en visualización")
                
#                 df_viz = df_viz_filtered
            
#             ax = axes[i]
            
#             real_data = df_viz[df_viz['Data_Type'] == 'Real']
#             random_data = df_viz[df_viz['Data_Type'] == 'Random']
            
#             ax.scatter(real_data['Feature_Value'], real_data['Predictions'], 
#                     color='blue', alpha=0.6, s=30, label='Real Data')
#             ax.scatter(random_data['Feature_Value'], random_data['Predictions'], 
#                     color='red', alpha=0.6, s=30, label='Random Data')
            
#             colors = {'Real': 'blue', 'Random': 'red'}
#             for data_type, color in colors.items():
#                 subset = df_viz[df_viz['Data_Type'] == data_type]
#                 if len(subset) > 1:
#                     lr = LinearRegression()
#                     X_trend = subset['Feature_Value'].values.reshape(-1, 1)
#                     y_trend = subset['Predictions'].values
                    
#                     if np.var(X_trend) > 1e-10:
#                         lr.fit(X_trend, y_trend)
                        
#                         x_line = np.linspace(subset['Feature_Value'].min(), 
#                                         subset['Feature_Value'].max(), 100)
#                         y_line = lr.predict(x_line.reshape(-1, 1))
                        
#                         try:
#                             r2 = lr.score(X_trend, y_trend)
#                             ax.plot(x_line, y_line, color=color, linestyle='--', 
#                                 alpha=0.8, linewidth=2, 
#                                 label=f'{data_type} Trend (R²={r2:.3f})')
#                         except:
#                             pass
            
#             ax.set_xlabel(f'{feature_name}')
#             ax.set_ylabel('Predicciones')
#             ax.set_title(f'{feature_name}\n({len(df_viz)} puntos) - Valores Originales')
#             ax.legend(fontsize=8)
#             ax.grid(True, alpha=0.3)
        
#         for i in range(len(feature_names), len(axes)):
#             axes[i].set_visible(False)
        
#         filter_status = "CON FILTRO" if filter_outliers else "SIN FILTRO"
#         fig.suptitle(f'Análisis por Variable - {model_name.upper()} ({architecture}) ({filter_status}) - Valores Originales', 
#                     fontsize=16, y=0.98)
#         plt.tight_layout()
#         plt.subplots_adjust(top=0.93)
        
#         suffix = "_filtered" if filter_outliers else "_original"
#         filename = f"{output_dir}/{model_name}_{architecture.lower()}_individual_features{suffix}.png"
#         plt.savefig(filename, dpi=300, bbox_inches='tight')
#         plt.close()
        
#         print(f"Gráfico guardado: {filename}")
        
#         return df_viz
#     def train_single_model(self, X_train, X_test, y_train, y_test, model_type, target_name, architecture='MLP'):
#         """Entrenar un solo modelo con arquitectura específica"""
#         print(f"\n=== {architecture} - {target_name.upper()} ===")
        
#         X_train_original = X_train.copy()
#         X_test_original = X_test.copy()
        
#         if architecture in ['RNN', 'LSTM', 'GRU', 'CNN1D', 'TRANSFORMER']:
#             sequence_length = min(10, len(X_train))
#             X_train_prepared = self.prepare_sequence_data(X_train, sequence_length)
#             X_test_prepared = self.prepare_sequence_data(X_test, sequence_length)
            
#             if len(y_train) > len(X_train_prepared):
#                 y_train = y_train[len(y_train) - len(X_train_prepared):]
#             if len(y_test) > len(X_test_prepared):
#                 y_test = y_test[len(y_test) - len(X_test_prepared):]
                
#             input_shape = X_train_prepared.shape[1:]
            
#         elif architecture == 'CNN2D':
#             X_train_prepared = self.prepare_cnn2d_data(X_train)
#             X_test_prepared = self.prepare_cnn2d_data(X_test)
#             input_shape = X_train_prepared.shape[1:]
            
#         else:
#             X_train_prepared = X_train_original
#             X_test_prepared = X_test_original
#             input_shape = X_train.shape[1]
        
#         model = None
#         if architecture == 'MLP':
#             model = self.create_mlp_model(input_shape, model_type)
#         elif architecture == 'RNN':
#             model = self.create_rnn_model(input_shape, model_type)
#         elif architecture == 'LSTM':
#             model = self.create_lstm_model(input_shape, model_type)
#         elif architecture == 'GRU':
#             model = self.create_gru_model(input_shape, model_type)
#         elif architecture == 'CNN1D':
#             model = self.create_cnn1d_model(input_shape, model_type)
#         elif architecture == 'CNN2D':
#             model = self.create_cnn2d_model(input_shape, model_type)
#         elif architecture == 'TRANSFORMER':
#             model = self.create_transformer_model(input_shape, model_type)
#         elif architecture == 'AUTOENCODER':
#             model = self.create_autoencoder_model(input_shape, model_type)
#         else:
#             print(f"Error: Arquitectura '{architecture}' no reconocida")
#             return None, None, 0, float('inf'), float('inf'), X_train_original, X_test_original
        
#         if model is None:
#             print(f"Error: No se pudo crear el modelo {architecture}")
#             return None, None, 0, float('inf'), float('inf'), X_train_original, X_test_original
        
#         early_stopping = keras.callbacks.EarlyStopping(
#             monitor='val_loss', patience=20, restore_best_weights=True
#         )
#         reduce_lr = keras.callbacks.ReduceLROnPlateau(
#             monitor='val_loss', factor=0.5, patience=10, min_lr=0.0001
#         )
        
#         epochs = 150 if architecture in ['TRANSFORMER', 'AUTOENCODER'] else 100
#         batch_size = 16 if architecture in ['RNN', 'LSTM', 'GRU'] else 32
        
#         try:
#             if architecture == 'AUTOENCODER':
#                 history = model.fit(
#                     X_train_prepared, [y_train, X_train_prepared],
#                     validation_split=0.2,
#                     epochs=epochs,
#                     batch_size=batch_size,
#                     callbacks=[early_stopping, reduce_lr],
#                     verbose=0
#                 )
#                 predictions = model.predict(X_test_prepared, verbose=0)[0].flatten()
#             else:
#                 history = model.fit(
#                     X_train_prepared, y_train,
#                     validation_split=0.2,
#                     epochs=epochs,
#                     batch_size=batch_size,
#                     callbacks=[early_stopping, reduce_lr],
#                     verbose=0
#                 )
#                 predictions = model.predict(X_test_prepared, verbose=0).flatten()
            
#             r2 = r2_score(y_test, predictions)
#             mae = mean_absolute_error(y_test, predictions)
#             rmse = np.sqrt(mean_squared_error(y_test, predictions))
            
#             print(f"R² Score: {r2:.4f}")
#             print(f"MAE: {mae:.4f}")
#             print(f"RMSE: {rmse:.4f}")
#             print(f"Parámetros: {model.count_params():,}")
#             print(f"Épocas entrenadas: {len(history.history['loss'])}")
            
#             return model, predictions, r2, mae, rmse, X_train_original, X_test_original
            
#         except Exception as e:
#             print(f"Error entrenando {architecture}: {e}")
#             import traceback
#             traceback.print_exc()
#             return None, None, 0, float('inf'), float('inf'), X_train_original, X_test_original
    
#     def analyze_business_patterns(self, df):
#         """Análisis de patrones de negocio"""
#         print(f"\n=== ANÁLISIS DE PATRONES DE NEGOCIO ===")
#         reinvested = df['BET_TOTAL'] > df['INITIAL_AMOUNT']
#         print(f"Sesiones con reinversión: {reinvested.sum()} ({reinvested.sum()/len(df)*100:.1f}%)")
        
#         net_winners = df['FINAL_AMOUNT'] > df['INITIAL_AMOUNT']
#         net_losers = df['FINAL_AMOUNT'] < df['INITIAL_AMOUNT']
        
#         print(f"Ganadores netos: {net_winners.sum()} ({net_winners.sum()/len(df)*100:.1f}%)")
#         print(f"Perdedores netos: {net_losers.sum()} ({net_losers.sum()/len(df)*100:.1f}%)")
        
#         win_bet_corr = np.corrcoef(df['WIN_TOTAL'], df['BET_TOTAL'])[0,1]
#         print(f"Correlación WIN vs BET: {win_bet_corr:.3f}")
        
#         print(f"Dataset shape: {df.shape}")
#         print(f"Rango TIEMPO: {df['TIME_ON_DEVICE_MIN'].min():.1f} - {df['TIME_ON_DEVICE_MIN'].max():.1f}")
#         print(f"Rango BET: {df['BET_TOTAL'].min():.1f} - {df['BET_TOTAL'].max():.1f}")
#         print(f"Rango WIN: {df['WIN_TOTAL'].min():.1f} - {df['WIN_TOTAL'].max():.1f}")
    
#     def train_and_visualize_all(self, df, architectures=['MLP'], filter_outliers=True, 
#                                outlier_method='iqr', filter_viz_outliers=True, 
#                                n_random_samples=1000, output_dir='multi_arch_results'):
#         """Entrenar múltiples arquitecturas y crear visualizaciones"""
#         print("=" * 80)
#         print("ENTRENANDO MÚLTIPLES ARQUITECTURAS CON VISUALIZACIONES")
#         print(f"Arquitecturas seleccionadas: {', '.join(architectures)}")
#         print("=" * 80)
        
#         if not os.path.exists(output_dir):
#             os.makedirs(output_dir)
        
#         viz_dir = f"{output_dir}/visualizations"
#         if not os.path.exists(viz_dir):
#             os.makedirs(viz_dir)
        
#         if filter_outliers:
#             df_processed = self.filter_outliers(df, method=outlier_method)
#         else:
#             df_processed = df.copy()
#             print(f"\nSin filtrado de outliers - Dataset: {len(df_processed)} registros")
        
#         self.analyze_business_patterns(df_processed)
        
#         X_base = df_processed[['INITIAL_AMOUNT','AVG_BET','Cluster','Weekday','Weekend','Month']]
#         y_tiempo = df_processed['TIME_ON_DEVICE_MIN']
#         y_bet = df_processed['BET_TOTAL'] 
#         y_win = df_processed['WIN_TOTAL']
        
#         X_base_train, X_base_test, y_tiempo_train, y_tiempo_test = train_test_split(
#             X_base, y_tiempo, test_size=0.2, random_state=42, 
#             stratify=pd.cut(y_tiempo, bins=5, labels=False)
#         )
        
#         train_idx, test_idx = X_base_train.index, X_base_test.index
        
#         all_results = {'tiempo': {}, 'bet': {}, 'win': {}}
#         all_visualizations = {'tiempo': {}, 'bet': {}, 'win': {}}
        
#         print(f"\n{'='*50}")
#         print("MODELOS PARA PREDICCIÓN DE TIEMPO")
#         print(f"{'='*50}")
        
#         X_tiempo_train, feature_names_tiempo = self.create_business_features(X_base_train)
#         X_tiempo_test, _ = self.create_business_features(X_base_test)
        
#         self.feature_names['tiempo'] = feature_names_tiempo
        
#         scaler_tiempo = RobustScaler()
#         X_tiempo_train_scaled = scaler_tiempo.fit_transform(X_tiempo_train)
#         X_tiempo_test_scaled = scaler_tiempo.transform(X_tiempo_test)
#         self.scalers['tiempo'] = scaler_tiempo
        
#         best_tiempo_r2 = -np.inf
#         best_tiempo_arch = None
#         best_tiempo_pred_train = None
#         best_tiempo_pred_test = None
        
#         for arch in architectures:
#             try:
#                 model, predictions, r2, mae, rmse, X_train_orig, X_test_orig = self.train_single_model(
#                     X_tiempo_train_scaled, X_tiempo_test_scaled, 
#                     y_tiempo_train.values, y_tiempo_test.values,
#                     'tiempo', 'tiempo', arch
#                 )
                
#                 if model is not None:
#                     all_results['tiempo'][arch] = {
#                         'R2': r2, 'MAE': mae, 'RMSE': rmse, 
#                         'model': model, 'predictions': predictions
#                     }
                    
#                     X_random_tiempo = self.generate_random_data(X_tiempo_train_scaled, n_random_samples)
                    
#                     X_tiempo_test_original = scaler_tiempo.inverse_transform(X_test_orig)
#                     X_random_tiempo_original = scaler_tiempo.inverse_transform(X_random_tiempo)
                    
#                     if arch in ['RNN', 'LSTM', 'GRU', 'CNN1D', 'TRANSFORMER']:
#                         X_random_prepared = self.prepare_sequence_data(X_random_tiempo, min(10, len(X_tiempo_train_scaled)))
#                     elif arch == 'CNN2D':
#                         X_random_prepared = self.prepare_cnn2d_data(X_random_tiempo)
#                     else:
#                         X_random_prepared = X_random_tiempo
                    
#                     if arch == 'AUTOENCODER':
#                         y_random_tiempo_pred = model.predict(X_random_prepared, verbose=0)[0].flatten()
#                     else:
#                         y_random_tiempo_pred = model.predict(X_random_prepared, verbose=0).flatten()
                    
#                     viz = self.create_individual_feature_visualizations(
#                         X_tiempo_test_scaled, y_tiempo_test.values, 
#                         X_random_tiempo, y_random_tiempo_pred, 
#                         X_tiempo_test_original, X_random_tiempo_original,
#                         feature_names_tiempo, 'tiempo', arch, viz_dir, filter_viz_outliers
#                     )
#                     all_visualizations['tiempo'][arch] = viz
                    
#                     if r2 > best_tiempo_r2:
#                         best_tiempo_r2 = r2
#                         best_tiempo_arch = arch
#                         if arch in ['RNN', 'LSTM', 'GRU', 'CNN1D', 'TRANSFORMER']:
#                             X_train_for_pred = self.prepare_sequence_data(X_tiempo_train_scaled, min(10, len(X_tiempo_train_scaled)))
#                             X_test_for_pred = self.prepare_sequence_data(X_tiempo_test_scaled, min(10, len(X_tiempo_test_scaled)))
#                         elif arch == 'CNN2D':
#                             X_train_for_pred = self.prepare_cnn2d_data(X_tiempo_train_scaled)
#                             X_test_for_pred = self.prepare_cnn2d_data(X_tiempo_test_scaled)
#                         else:
#                             X_train_for_pred = X_tiempo_train_scaled
#                             X_test_for_pred = X_tiempo_test_scaled
                        
#                         if arch == 'AUTOENCODER':
#                             best_tiempo_pred_train = model.predict(X_train_for_pred, verbose=0)[0].flatten()
#                             best_tiempo_pred_test = model.predict(X_test_for_pred, verbose=0)[0].flatten()
#                         else:
#                             best_tiempo_pred_train = model.predict(X_train_for_pred, verbose=0).flatten()
#                             best_tiempo_pred_test = model.predict(X_test_for_pred, verbose=0).flatten()
                        
#             except Exception as e:
#                 print(f"Error con {arch} para tiempo: {e}")
#                 import traceback
#                 traceback.print_exc()
#                 continue
        
#         print(f"\n🏆 Mejor modelo TIEMPO: {best_tiempo_arch} (R² = {best_tiempo_r2:.4f})")
        
#         print(f"\n{'='*50}")
#         print("MODELOS PARA PREDICCIÓN DE BET TOTAL")
#         print(f"{'='*50}")
        
#         X_bet_train, feature_names_bet = self.create_business_features(X_base_train, best_tiempo_pred_train)
#         X_bet_test, _ = self.create_business_features(X_base_test, best_tiempo_pred_test)
        
#         self.feature_names['bet'] = feature_names_bet
        
#         scaler_bet = RobustScaler()
#         X_bet_train_scaled = scaler_bet.fit_transform(X_bet_train)
#         X_bet_test_scaled = scaler_bet.transform(X_bet_test)
#         self.scalers['bet'] = scaler_bet
        
#         best_bet_r2 = -np.inf
#         best_bet_arch = None
#         best_bet_pred_test = None
        
#         for arch in architectures:
#             try:
#                 model, predictions, r2, mae, rmse, X_train_orig, X_test_orig = self.train_single_model(
#                     X_bet_train_scaled, X_bet_test_scaled,
#                     y_bet.loc[train_idx].values, y_bet.loc[test_idx].values,
#                     'bet', 'bet_total', arch
#                 )
                
#                 if model is not None:
#                     all_results['bet'][arch] = {
#                         'R2': r2, 'MAE': mae, 'RMSE': rmse, 
#                         'model': model, 'predictions': predictions
#                     }
                    
#                     X_random_bet = self.generate_random_data(X_bet_train_scaled, n_random_samples)
                    
#                     X_bet_test_original = scaler_bet.inverse_transform(X_test_orig)
#                     X_random_bet_original = scaler_bet.inverse_transform(X_random_bet)
                    
#                     if arch in ['RNN', 'LSTM', 'GRU', 'CNN1D', 'TRANSFORMER']:
#                         X_random_prepared = self.prepare_sequence_data(X_random_bet, min(10, len(X_bet_train_scaled)))
#                     elif arch == 'CNN2D':
#                         X_random_prepared = self.prepare_cnn2d_data(X_random_bet)
#                     else:
#                         X_random_prepared = X_random_bet
                    
#                     if arch == 'AUTOENCODER':
#                         y_random_bet_pred = model.predict(X_random_prepared, verbose=0)[0].flatten()
#                     else:
#                         y_random_bet_pred = model.predict(X_random_prepared, verbose=0).flatten()
                    
#                     viz = self.create_individual_feature_visualizations(
#                         X_bet_test_scaled, y_bet.loc[test_idx].values,
#                         X_random_bet, y_random_bet_pred,
#                         X_bet_test_original, X_random_bet_original,
#                         feature_names_bet, 'bet', arch, viz_dir, filter_viz_outliers
#                     )
#                     all_visualizations['bet'][arch] = viz
                    
#                     if r2 > best_bet_r2:
#                         best_bet_r2 = r2
#                         best_bet_arch = arch
                        
#                         if arch in ['RNN', 'LSTM', 'GRU', 'CNN1D', 'TRANSFORMER']:
#                             X_test_for_pred = self.prepare_sequence_data(X_bet_test_scaled, min(10, len(X_bet_test_scaled)))
#                         elif arch == 'CNN2D':
#                             X_test_for_pred = self.prepare_cnn2d_data(X_bet_test_scaled)
#                         else:
#                             X_test_for_pred = X_bet_test_scaled
                        
#                         if arch == 'AUTOENCODER':
#                             best_bet_pred_test = model.predict(X_test_for_pred, verbose=0)[0].flatten()
#                         else:
#                             best_bet_pred_test = model.predict(X_test_for_pred, verbose=0).flatten()
                        
#             except Exception as e:
#                 print(f"Error con {arch} para bet: {e}")
#                 import traceback
#                 traceback.print_exc()
#                 continue
        
#         print(f"\n🏆 Mejor modelo BET: {best_bet_arch} (R² = {best_bet_r2:.4f})")
        
#         print(f"\n{'='*50}")
#         print("MODELOS PARA PREDICCIÓN DE WIN TOTAL")
#         print(f"{'='*50}")
        
#         best_bet_model = all_results['bet'][best_bet_arch]['model']
#         if best_bet_arch in ['RNN', 'LSTM', 'GRU', 'CNN1D', 'TRANSFORMER']:
#             X_bet_train_prep = self.prepare_sequence_data(X_bet_train_scaled, min(10, len(X_bet_train_scaled)))
#         elif best_bet_arch == 'CNN2D':
#             X_bet_train_prep = self.prepare_cnn2d_data(X_bet_train_scaled)
#         else:
#             X_bet_train_prep = X_bet_train_scaled
        
#         if best_bet_arch == 'AUTOENCODER':
#             best_bet_pred_train = best_bet_model.predict(X_bet_train_prep, verbose=0)[0].flatten()
#         else:
#             best_bet_pred_train = best_bet_model.predict(X_bet_train_prep, verbose=0).flatten()
        
#         X_win_train, feature_names_win = self.create_business_features(
#             X_base_train, best_tiempo_pred_train, best_bet_pred_train
#         )
#         X_win_test, _ = self.create_business_features(
#             X_base_test, best_tiempo_pred_test, best_bet_pred_test
#         )
        
#         self.feature_names['win'] = feature_names_win
        
#         scaler_win = RobustScaler()
#         X_win_train_scaled = scaler_win.fit_transform(X_win_train)
#         X_win_test_scaled = scaler_win.transform(X_win_test)
#         self.scalers['win'] = scaler_win
        
#         best_win_r2 = -np.inf
#         best_win_arch = None
        
#         for arch in architectures:
#             try:
#                 model, predictions, r2, mae, rmse, X_train_orig, X_test_orig = self.train_single_model(
#                     X_win_train_scaled, X_win_test_scaled,
#                     y_win.loc[train_idx].values, y_win.loc[test_idx].values,
#                     'win', 'win_total', arch
#                 )
                
#                 if model is not None:
#                     all_results['win'][arch] = {
#                         'R2': r2, 'MAE': mae, 'RMSE': rmse, 
#                         'model': model, 'predictions': predictions
#                     }
                    
#                     X_random_win = self.generate_random_data(X_win_train_scaled, n_random_samples)
                    
#                     X_win_test_original = scaler_win.inverse_transform(X_test_orig)
#                     X_random_win_original = scaler_win.inverse_transform(X_random_win)
                    
#                     if arch in ['RNN', 'LSTM', 'GRU', 'CNN1D', 'TRANSFORMER']:
#                         X_random_prepared = self.prepare_sequence_data(X_random_win, min(10, len(X_win_train_scaled)))
#                     elif arch == 'CNN2D':
#                         X_random_prepared = self.prepare_cnn2d_data(X_random_win)
#                     else:
#                         X_random_prepared = X_random_win
                    
#                     if arch == 'AUTOENCODER':
#                         y_random_win_pred = model.predict(X_random_prepared, verbose=0)[0].flatten()
#                     else:
#                         y_random_win_pred = model.predict(X_random_prepared, verbose=0).flatten()
                    
#                     viz = self.create_individual_feature_visualizations(
#                         X_win_test_scaled, y_win.loc[test_idx].values,
#                         X_random_win, y_random_win_pred,
#                         X_win_test_original, X_random_win_original,
#                         feature_names_win, 'win', arch, viz_dir, filter_viz_outliers
#                     )
#                     all_visualizations['win'][arch] = viz
                    
#                     if r2 > best_win_r2:
#                         best_win_r2 = r2
#                         best_win_arch = arch
                        
#             except Exception as e:
#                 print(f"Error con {arch} para win: {e}")
#                 import traceback
#                 traceback.print_exc()
#                 continue
        
#         if best_win_arch:
#             print(f"\n🏆 Mejor modelo WIN: {best_win_arch} (R² = {best_win_r2:.4f})")
        
#         print(f"\n{'='*80}")
#         print("RESUMEN COMPARATIVO DE TODAS LAS ARQUITECTURAS")
#         print(f"{'='*80}")
        
#         comparison_data = []
#         for target in ['tiempo', 'bet', 'win']:
#             for arch in architectures:
#                 if arch in all_results[target]:
#                     result = all_results[target][arch]
#                     comparison_data.append({
#                         'Target': target.upper(),
#                         'Architecture': arch,
#                         'R2_Score': result['R2'],
#                         'MAE': result['MAE'],
#                         'RMSE': result['RMSE'],
#                         'N_Features': len(self.feature_names[target]),
#                         'Features': ', '.join(self.feature_names[target])
#                     })
        
#         comparison_df = pd.DataFrame(comparison_data)
#         comparison_filename = f"{output_dir}/architecture_comparison.csv"
#         comparison_df.to_csv(comparison_filename, index=False)
#         print(f"✓ Comparación guardada: {comparison_filename}")
        
#         print(f"\n🏆 MEJORES MODELOS POR OBJETIVO:")
#         print(f"   TIEMPO: {best_tiempo_arch} (R² = {best_tiempo_r2:.4f})")
#         print(f"   BET: {best_bet_arch} (R² = {best_bet_r2:.4f})")
#         if best_win_arch:
#             print(f"   WIN: {best_win_arch} (R² = {best_win_r2:.4f})")
        
#         arch_rankings = {}
#         for arch in architectures:
#             r2_scores = []
#             for target in ['tiempo', 'bet', 'win']:
#                 if arch in all_results[target]:
#                     r2_scores.append(all_results[target][arch]['R2'])
#             if r2_scores:
#                 arch_rankings[arch] = np.mean(r2_scores)
        
#         print(f"\n📊 RANKING DE ARQUITECTURAS (R² promedio):")
#         for i, (arch, avg_r2) in enumerate(sorted(arch_rankings.items(), key=lambda x: x[1], reverse=True), 1):
#             print(f"   {i}. {arch}: {avg_r2:.4f}")
        
#         print(f"\n📁 Resultados guardados en: '{output_dir}'")
#         print(f"📁 Visualizaciones guardadas en: '{viz_dir}'")
        
#         return {
#             'results': all_results,
#             'comparison': comparison_df,
#             'visualizations': all_visualizations,
#             'best_models': {
#                 'tiempo': best_tiempo_arch,
#                 'bet': best_bet_arch,
#                 'win': best_win_arch
#             }
#         }








# # ============================================================================
# # FIX 2: Update create_individual_feature_visualizations to handle length mismatches
# # ============================================================================

# def create_individual_feature_visualizations(self, X_real_scaled, y_real, X_random_scaled, y_pred_random, 
#                                            X_real_original, X_random_original, feature_names, model_name, 
#                                            architecture, output_dir='visualizations', filter_outliers=True):
#     """Crear visualizaciones individuales para cada variable con valores sin escalar"""
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)
    
#     n_features = X_real_original.shape[1]
#     print(f"\n=== CREANDO VISUALIZACIONES PARA {n_features} VARIABLES - {model_name.upper()} ({architecture}) ===")
    
#     # CRITICAL FIX: Ensure all arrays have the same length
#     min_len = min(len(X_real_original), len(y_real))
#     X_real_original = X_real_original[:min_len]
#     y_real = y_real[:min_len]
    
#     # Also ensure random data matches
#     min_random_len = min(len(X_random_original), len(y_pred_random))
#     X_random_original = X_random_original[:min_random_len]
#     y_pred_random = y_pred_random[:min_random_len]
    
#     print(f"Real data points: {len(X_real_original)}, Random data points: {len(X_random_original)}")
    
#     fig, axes = plt.subplots(2, (n_features + 1) // 2, figsize=(6 * ((n_features + 1) // 2), 12))
#     if n_features == 1:
#         axes = [axes]
#     elif (n_features + 1) // 2 == 1:
#         axes = axes.reshape(-1, 1)
#     axes = axes.flatten()
    
#     for i, feature_name in enumerate(feature_names):
#         if i >= len(axes):
#             break
            
#         df_viz = pd.DataFrame({
#             'Feature_Value': np.concatenate([X_real_original[:, i], X_random_original[:, i]]),
#             'Predictions': np.concatenate([y_real, y_pred_random]),
#             'Data_Type': ['Real'] * len(X_real_original) + ['Random'] * len(X_random_original)
#         })
        
#         if filter_outliers:
#             feature_p5 = df_viz['Feature_Value'].quantile(0.05)
#             feature_p95 = df_viz['Feature_Value'].quantile(0.95)
#             pred_p5 = df_viz['Predictions'].quantile(0.05)
#             pred_p95 = df_viz['Predictions'].quantile(0.95)
            
#             mask = (
#                 (df_viz['Feature_Value'] >= feature_p5) & 
#                 (df_viz['Feature_Value'] <= feature_p95) &
#                 (df_viz['Predictions'] >= pred_p5) &
#                 (df_viz['Predictions'] <= pred_p95)
#             )
            
#             df_viz_filtered = df_viz[mask].copy()
#             removed_points = len(df_viz) - len(df_viz_filtered)
            
#             if removed_points > 0:
#                 print(f"{feature_name}: Removidos {removed_points} outliers en visualización")
            
#             df_viz = df_viz_filtered
        
#         ax = axes[i]
        
#         real_data = df_viz[df_viz['Data_Type'] == 'Real']
#         random_data = df_viz[df_viz['Data_Type'] == 'Random']
        
#         ax.scatter(real_data['Feature_Value'], real_data['Predictions'], 
#                   color='blue', alpha=0.6, s=30, label='Real Data')
#         ax.scatter(random_data['Feature_Value'], random_data['Predictions'], 
#                   color='red', alpha=0.6, s=30, label='Random Data')
        
#         colors = {'Real': 'blue', 'Random': 'red'}
#         for data_type, color in colors.items():
#             subset = df_viz[df_viz['Data_Type'] == data_type]
#             if len(subset) > 1:
#                 lr = LinearRegression()
#                 X_trend = subset['Feature_Value'].values.reshape(-1, 1)
#                 y_trend = subset['Predictions'].values
                
#                 if np.var(X_trend) > 1e-10:
#                     lr.fit(X_trend, y_trend)
                    
#                     x_line = np.linspace(subset['Feature_Value'].min(), 
#                                        subset['Feature_Value'].max(), 100)
#                     y_line = lr.predict(x_line.reshape(-1, 1))
                    
#                     try:
#                         r2 = lr.score(X_trend, y_trend)
#                         ax.plot(x_line, y_line, color=color, linestyle='--', 
#                                alpha=0.8, linewidth=2, 
#                                label=f'{data_type} Trend (R²={r2:.3f})')
#                     except:
#                         pass
        
#         ax.set_xlabel(f'{feature_name}')
#         ax.set_ylabel('Predicciones')
#         ax.set_title(f'{feature_name}\n({len(df_viz)} puntos) - Valores Originales')
#         ax.legend(fontsize=8)
#         ax.grid(True, alpha=0.3)
    
#     for i in range(len(feature_names), len(axes)):
#         axes[i].set_visible(False)
    
#     filter_status = "CON FILTRO" if filter_outliers else "SIN FILTRO"
#     fig.suptitle(f'Análisis por Variable - {model_name.upper()} ({architecture}) ({filter_status}) - Valores Originales', 
#                  fontsize=16, y=0.98)
#     plt.tight_layout()
#     plt.subplots_adjust(top=0.93)
    
#     suffix = "_filtered" if filter_outliers else "_original"
#     filename = f"{output_dir}/{model_name}_{architecture.lower()}_individual_features{suffix}.png"
#     plt.savefig(filename, dpi=300, bbox_inches='tight')
#     plt.close()
    
#     print(f"Gráfico guardado: {filename}")
    
#     return df_viz


In [11]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import warnings
warnings.filterwarnings('ignore')

class MultiArchitectureCasinoML:
    def __init__(self):
        self.scalers = {}
        self.models = {}
        self.feature_names = {}
        self.model_types = ['MLP', 'RNN', 'LSTM', 'GRU', 'CNN1D', 'CNN2D', 'TRANSFORMER', 'AUTOENCODER']
        
    def filter_outliers(self, df, method='iqr', factor=1.5):
        """Filtrar outliers del dataset"""
        print(f"\n=== FILTRADO DE OUTLIERS (Método: {method.upper()}) ===")
        print(f"Dataset original: {len(df)} registros")
        
        df_filtered = df.copy()
        numeric_cols = ['BET_TOTAL', 'WIN_TOTAL', 'TIME_ON_DEVICE_MIN', 'INITIAL_AMOUNT']
        
        if method == 'iqr':
            for col in numeric_cols:
                if col in df_filtered.columns:
                    Q1 = df_filtered[col].quantile(0.25)
                    Q3 = df_filtered[col].quantile(0.75)
                    IQR = Q3 - Q1
                    lower_bound = Q1 - factor * IQR
                    upper_bound = Q3 + factor * IQR
                    
                    before_count = len(df_filtered)
                    df_filtered = df_filtered[
                        (df_filtered[col] >= lower_bound) & 
                        (df_filtered[col] <= upper_bound)
                    ]
                    removed = before_count - len(df_filtered)
                    if removed > 0:
                        print(f"{col}: Removidos {removed} outliers (rango: {lower_bound:.1f} - {upper_bound:.1f})")
        
        elif method == 'zscore':
            from scipy import stats
            for col in numeric_cols:
                if col in df_filtered.columns:
                    z_scores = np.abs(stats.zscore(df_filtered[col]))
                    before_count = len(df_filtered)
                    df_filtered = df_filtered[z_scores < 3]
                    removed = before_count - len(df_filtered)
                    if removed > 0:
                        print(f"{col}: Removidos {removed} outliers (z-score >= 3)")
        
        elif method == 'percentile':
            for col in numeric_cols:
                if col in df_filtered.columns:
                    lower_percentile = df_filtered[col].quantile(0.02)
                    upper_percentile = df_filtered[col].quantile(0.98)
                    before_count = len(df_filtered)
                    df_filtered = df_filtered[
                        (df_filtered[col] >= lower_percentile) & 
                        (df_filtered[col] <= upper_percentile)
                    ]
                    removed = before_count - len(df_filtered)
                    if removed > 0:
                        print(f"{col}: Removidos {removed} outliers (percentiles 2-98)")
        
        print(f"Dataset final: {len(df_filtered)} registros")
        print(f"Total removidos: {len(df) - len(df_filtered)} ({(len(df) - len(df_filtered))/len(df)*100:.1f}%)")
        
        return df_filtered
    
    def create_business_features(self, df, tiempo_pred=None, bet_pred=None):
        """Crear features que reflejen la lógica real del negocio de casino"""
        features = df[['INITIAL_AMOUNT', 'AVG_BET', 'Cluster']].copy()
        feature_names = ['INITIAL_AMOUNT', 'AVG_BET', 'Cluster']
        
        if tiempo_pred is not None:
            # FIX: Ensure tiempo_pred matches the length of df
            if len(tiempo_pred) != len(df):
                print(f"WARNING: tiempo_pred length ({len(tiempo_pred)}) != df length ({len(df)})")
                # Truncate or pad to match
                if len(tiempo_pred) > len(df):
                    tiempo_pred = tiempo_pred[:len(df)]
                else:
                    # Pad with mean value
                    padding = np.full(len(df) - len(tiempo_pred), np.mean(tiempo_pred))
                    tiempo_pred = np.concatenate([tiempo_pred, padding])
            
            features['tiempo_pred'] = tiempo_pred
            feature_names.append('tiempo_pred')
            
        if bet_pred is not None:
            # FIX: Ensure bet_pred matches the length of df
            if len(bet_pred) != len(df):
                print(f"WARNING: bet_pred length ({len(bet_pred)}) != df length ({len(df)})")
                if len(bet_pred) > len(df):
                    bet_pred = bet_pred[:len(df)]
                else:
                    padding = np.full(len(df) - len(bet_pred), np.mean(bet_pred))
                    bet_pred = np.concatenate([bet_pred, padding])
            
            features['bet_pred'] = bet_pred
            features['total_money_handled'] = bet_pred
            features['house_edge_effect'] = bet_pred * 0.05
            features['money_multiplier'] = bet_pred / (df['INITIAL_AMOUNT'].values + 1)
            features['reinvestment_indicator'] = np.where(bet_pred > df['INITIAL_AMOUNT'].values, 1, 0)
            features['excess_betting'] = np.maximum(0, bet_pred - df['INITIAL_AMOUNT'].values)
            features['money_at_risk'] = np.minimum(bet_pred, df['INITIAL_AMOUNT'].values)
            features['cluster_risk_adjusted'] = df['Cluster'].values * features['money_multiplier']
            
            feature_names.extend(['bet_pred', 'total_money_handled', 'house_edge_effect', 
                                'money_multiplier', 'reinvestment_indicator', 
                                'excess_betting', 'money_at_risk', 'cluster_risk_adjusted'])
            
        return features, feature_names
    
    def prepare_sequence_data(self, X, sequence_length=10):
        """Preparar datos para modelos secuenciales"""
        if len(X) < sequence_length:
            X_padded = np.pad(X, ((sequence_length - len(X), 0), (0, 0)), mode='edge')
            X = X_padded
        
        X_seq = []
        for i in range(len(X) - sequence_length + 1):
            X_seq.append(X[i:(i + sequence_length)])
        
        if len(X_seq) == 0:
            if len(X) == 0:
                return np.zeros((1, sequence_length, X.shape[1] if len(X.shape) > 1 else 1))
            X_seq = [np.pad(X, ((max(0, sequence_length - len(X)), 0), (0, 0)), mode='edge')[-sequence_length:]]
            
        return np.array(X_seq)
    
    def prepare_cnn2d_data(self, X, height=4, width=None):
        """Preparar datos para CNN2D"""
        n_features = X.shape[1]
        
        if width is None:
            width = n_features // height
            if n_features % height != 0:
                pad_size = height - (n_features % height)
                X = np.pad(X, ((0, 0), (0, pad_size)), mode='constant')
                n_features = X.shape[1]
                width = n_features // height
        
        X_2d = X.reshape(X.shape[0], height, width, 1)
        return X_2d
    
    def create_mlp_model(self, input_shape, model_type='tiempo'):
        """Crear modelo MLP"""
        model = keras.Sequential()
        
        if model_type == 'tiempo':
            model.add(layers.Dense(128, activation='relu', input_shape=(input_shape,)))
            model.add(layers.BatchNormalization())
            model.add(layers.Dropout(0.3))
            model.add(layers.Dense(64, activation='relu'))
            model.add(layers.BatchNormalization())
            model.add(layers.Dropout(0.2))
            model.add(layers.Dense(32, activation='relu'))
            model.add(layers.Dropout(0.1))
            model.add(layers.Dense(1, activation='linear'))
            
        elif model_type == 'bet':
            model.add(layers.Dense(256, activation='relu', input_shape=(input_shape,)))
            model.add(layers.BatchNormalization())
            model.add(layers.Dropout(0.4))
            model.add(layers.Dense(128, activation='relu'))
            model.add(layers.BatchNormalization())
            model.add(layers.Dropout(0.3))
            model.add(layers.Dense(64, activation='relu'))
            model.add(layers.BatchNormalization())
            model.add(layers.Dropout(0.2))
            model.add(layers.Dense(32, activation='relu'))
            model.add(layers.Dropout(0.1))
            model.add(layers.Dense(1, activation='linear'))
            
        elif model_type == 'win':
            model.add(layers.Dense(512, activation='relu', input_shape=(input_shape,)))
            model.add(layers.BatchNormalization())
            model.add(layers.Dropout(0.4))
            model.add(layers.Dense(256, activation='relu'))
            model.add(layers.BatchNormalization())
            model.add(layers.Dropout(0.3))
            model.add(layers.Dense(128, activation='relu'))
            model.add(layers.BatchNormalization())
            model.add(layers.Dropout(0.3))
            model.add(layers.Dense(64, activation='relu'))
            model.add(layers.BatchNormalization())
            model.add(layers.Dropout(0.2))
            model.add(layers.Dense(32, activation='relu'))
            model.add(layers.Dropout(0.1))
            model.add(layers.Dense(1, activation='linear'))
        
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )
        
        return model
    
    def create_rnn_model(self, input_shape, model_type='tiempo'):
        """Crear modelo RNN"""
        model = keras.Sequential()
        
        if model_type == 'tiempo':
            model.add(layers.SimpleRNN(64, return_sequences=True, input_shape=input_shape))
            model.add(layers.Dropout(0.3))
            model.add(layers.SimpleRNN(32, return_sequences=False))
            model.add(layers.Dropout(0.2))
            model.add(layers.Dense(16, activation='relu'))
            model.add(layers.Dense(1, activation='linear'))
            
        elif model_type == 'bet':
            model.add(layers.SimpleRNN(128, return_sequences=True, input_shape=input_shape))
            model.add(layers.Dropout(0.4))
            model.add(layers.SimpleRNN(64, return_sequences=True))
            model.add(layers.Dropout(0.3))
            model.add(layers.SimpleRNN(32, return_sequences=False))
            model.add(layers.Dropout(0.2))
            model.add(layers.Dense(16, activation='relu'))
            model.add(layers.Dense(1, activation='linear'))
            
        elif model_type == 'win':
            model.add(layers.SimpleRNN(256, return_sequences=True, input_shape=input_shape))
            model.add(layers.Dropout(0.4))
            model.add(layers.SimpleRNN(128, return_sequences=True))
            model.add(layers.Dropout(0.3))
            model.add(layers.SimpleRNN(64, return_sequences=False))
            model.add(layers.Dropout(0.2))
            model.add(layers.Dense(32, activation='relu'))
            model.add(layers.Dense(16, activation='relu'))
            model.add(layers.Dense(1, activation='linear'))
        
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )
        
        return model
    
    def create_lstm_model(self, input_shape, model_type='tiempo'):
        """Crear modelo LSTM"""
        model = keras.Sequential()
        
        if model_type == 'tiempo':
            model.add(layers.LSTM(64, return_sequences=True, input_shape=input_shape))
            model.add(layers.Dropout(0.3))
            model.add(layers.LSTM(32, return_sequences=False))
            model.add(layers.Dropout(0.2))
            model.add(layers.Dense(16, activation='relu'))
            model.add(layers.Dense(1, activation='linear'))
            
        elif model_type == 'bet':
            model.add(layers.LSTM(128, return_sequences=True, input_shape=input_shape))
            model.add(layers.Dropout(0.4))
            model.add(layers.LSTM(64, return_sequences=True))
            model.add(layers.Dropout(0.3))
            model.add(layers.LSTM(32, return_sequences=False))
            model.add(layers.Dropout(0.2))
            model.add(layers.Dense(16, activation='relu'))
            model.add(layers.Dense(1, activation='linear'))
            
        elif model_type == 'win':
            model.add(layers.LSTM(256, return_sequences=True, input_shape=input_shape))
            model.add(layers.Dropout(0.4))
            model.add(layers.LSTM(128, return_sequences=True))
            model.add(layers.Dropout(0.3))
            model.add(layers.LSTM(64, return_sequences=False))
            model.add(layers.Dropout(0.2))
            model.add(layers.Dense(32, activation='relu'))
            model.add(layers.Dense(16, activation='relu'))
            model.add(layers.Dense(1, activation='linear'))
        
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )
        
        return model
    
    def create_gru_model(self, input_shape, model_type='tiempo'):
        """Crear modelo GRU"""
        model = keras.Sequential()
        
        if model_type == 'tiempo':
            model.add(layers.GRU(64, return_sequences=True, input_shape=input_shape))
            model.add(layers.Dropout(0.3))
            model.add(layers.GRU(32, return_sequences=False))
            model.add(layers.Dropout(0.2))
            model.add(layers.Dense(16, activation='relu'))
            model.add(layers.Dense(1, activation='linear'))
            
        elif model_type == 'bet':
            model.add(layers.GRU(128, return_sequences=True, input_shape=input_shape))
            model.add(layers.Dropout(0.4))
            model.add(layers.GRU(64, return_sequences=True))
            model.add(layers.Dropout(0.3))
            model.add(layers.GRU(32, return_sequences=False))
            model.add(layers.Dropout(0.2))
            model.add(layers.Dense(16, activation='relu'))
            model.add(layers.Dense(1, activation='linear'))
            
        elif model_type == 'win':
            model.add(layers.GRU(256, return_sequences=True, input_shape=input_shape))
            model.add(layers.Dropout(0.4))
            model.add(layers.GRU(128, return_sequences=True))
            model.add(layers.Dropout(0.3))
            model.add(layers.GRU(64, return_sequences=False))
            model.add(layers.Dropout(0.2))
            model.add(layers.Dense(32, activation='relu'))
            model.add(layers.Dense(16, activation='relu'))
            model.add(layers.Dense(1, activation='linear'))
        
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )
        
        return model
    
    def create_cnn1d_model(self, input_shape, model_type='tiempo'):
        """Crear modelo CNN 1D"""
        model = keras.Sequential()
        
        if model_type == 'tiempo':
            model.add(layers.Conv1D(64, 3, activation='relu', input_shape=input_shape))
            model.add(layers.BatchNormalization())
            model.add(layers.MaxPooling1D(2))
            model.add(layers.Conv1D(32, 3, activation='relu'))
            model.add(layers.BatchNormalization())
            model.add(layers.GlobalMaxPooling1D())
            model.add(layers.Dense(50, activation='relu'))
            model.add(layers.Dropout(0.3))
            model.add(layers.Dense(1, activation='linear'))
            
        elif model_type == 'bet':
            model.add(layers.Conv1D(128, 3, activation='relu', input_shape=input_shape))
            model.add(layers.BatchNormalization())
            model.add(layers.MaxPooling1D(2))
            model.add(layers.Conv1D(64, 3, activation='relu'))
            model.add(layers.BatchNormalization())
            model.add(layers.Conv1D(32, 3, activation='relu'))
            model.add(layers.GlobalMaxPooling1D())
            model.add(layers.Dense(100, activation='relu'))
            model.add(layers.Dropout(0.4))
            model.add(layers.Dense(50, activation='relu'))
            model.add(layers.Dropout(0.3))
            model.add(layers.Dense(1, activation='linear'))
            
        elif model_type == 'win':
            model.add(layers.Conv1D(256, 3, activation='relu', input_shape=input_shape))
            model.add(layers.BatchNormalization())
            model.add(layers.MaxPooling1D(2))
            model.add(layers.Conv1D(128, 3, activation='relu'))
            model.add(layers.BatchNormalization())
            model.add(layers.Conv1D(64, 3, activation='relu'))
            model.add(layers.BatchNormalization())
            model.add(layers.GlobalMaxPooling1D())
            model.add(layers.Dense(200, activation='relu'))
            model.add(layers.Dropout(0.4))
            model.add(layers.Dense(100, activation='relu'))
            model.add(layers.Dropout(0.3))
            model.add(layers.Dense(50, activation='relu'))
            model.add(layers.Dropout(0.2))
            model.add(layers.Dense(1, activation='linear'))
        
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )
        
        return model
    
    def create_cnn2d_model(self, input_shape, model_type='tiempo'):
        """Crear modelo CNN 2D"""
        model = keras.Sequential()
        
        if model_type == 'tiempo':
            model.add(layers.Conv2D(32, (2, 2), activation='relu', input_shape=input_shape))
            model.add(layers.BatchNormalization())
            model.add(layers.Conv2D(16, (2, 2), activation='relu'))
            model.add(layers.BatchNormalization())
            model.add(layers.GlobalMaxPooling2D())
            model.add(layers.Dense(32, activation='relu'))
            model.add(layers.Dropout(0.3))
            model.add(layers.Dense(1, activation='linear'))
            
        elif model_type == 'bet':
            model.add(layers.Conv2D(64, (2, 2), activation='relu', input_shape=input_shape))
            model.add(layers.BatchNormalization())
            model.add(layers.Conv2D(32, (2, 2), activation='relu'))
            model.add(layers.BatchNormalization())
            model.add(layers.Conv2D(16, (2, 2), activation='relu'))
            model.add(layers.GlobalMaxPooling2D())
            model.add(layers.Dense(64, activation='relu'))
            model.add(layers.Dropout(0.4))
            model.add(layers.Dense(32, activation='relu'))
            model.add(layers.Dropout(0.3))
            model.add(layers.Dense(1, activation='linear'))
            
        elif model_type == 'win':
            model.add(layers.Conv2D(128, (2, 2), activation='relu', input_shape=input_shape))
            model.add(layers.BatchNormalization())
            model.add(layers.Conv2D(64, (2, 2), activation='relu'))
            model.add(layers.BatchNormalization())
            model.add(layers.Conv2D(32, (2, 2), activation='relu'))
            model.add(layers.BatchNormalization())
            model.add(layers.GlobalMaxPooling2D())
            model.add(layers.Dense(128, activation='relu'))
            model.add(layers.Dropout(0.4))
            model.add(layers.Dense(64, activation='relu'))
            model.add(layers.Dropout(0.3))
            model.add(layers.Dense(32, activation='relu'))
            model.add(layers.Dropout(0.2))
            model.add(layers.Dense(1, activation='linear'))
        
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )
        
        return model
    
    def create_transformer_model(self, input_shape, model_type='tiempo'):
        """Crear modelo Transformer"""
        inputs = keras.Input(shape=input_shape)
        
        if model_type == 'tiempo':
            attention = layers.MultiHeadAttention(num_heads=4, key_dim=16)(inputs, inputs)
            attention = layers.Dropout(0.3)(attention)
            attention = layers.LayerNormalization()(attention + inputs)
            
            ff = layers.Dense(64, activation='relu')(attention)
            ff = layers.Dropout(0.3)(ff)
            ff = layers.Dense(input_shape[-1])(ff)
            ff = layers.LayerNormalization()(ff + attention)
            
        elif model_type == 'win':
            attention = layers.MultiHeadAttention(num_heads=8, key_dim=64)(inputs, inputs)
            attention = layers.Dropout(0.4)(attention)
            attention = layers.LayerNormalization()(attention + inputs)
            
            ff = layers.Dense(256, activation='relu')(attention)
            ff = layers.Dropout(0.4)(ff)
            ff = layers.Dense(128, activation='relu')(ff)
            ff = layers.Dropout(0.3)(ff)
            ff = layers.Dense(64, activation='relu')(ff)
            ff = layers.Dropout(0.2)(ff)
            ff = layers.Dense(input_shape[-1])(ff)
            ff = layers.LayerNormalization()(ff + attention)
        
        pooled = layers.GlobalAveragePooling1D()(ff)
        outputs = layers.Dense(1, activation='linear')(pooled)
        
        model = keras.Model(inputs, outputs)
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )
        
        return model
    
    def create_autoencoder_model(self, input_shape, model_type='tiempo'):
        """Crear modelo Autoencoder"""
        input_layer = keras.Input(shape=(input_shape,))
        
        if model_type == 'tiempo':
            encoded = layers.Dense(64, activation='relu')(input_layer)
            encoded = layers.BatchNormalization()(encoded)
            encoded = layers.Dropout(0.3)(encoded)
            encoded = layers.Dense(32, activation='relu')(encoded)
            encoded = layers.BatchNormalization()(encoded)
            encoded = layers.Dropout(0.2)(encoded)
            encoded = layers.Dense(16, activation='relu')(encoded)
            
            decoded = layers.Dense(32, activation='relu')(encoded)
            decoded = layers.BatchNormalization()(decoded)
            decoded = layers.Dropout(0.2)(decoded)
            decoded = layers.Dense(64, activation='relu')(decoded)
            decoded = layers.BatchNormalization()(decoded)
            decoded = layers.Dropout(0.3)(decoded)
            decoded = layers.Dense(input_shape, activation='linear')(decoded)
            
            predictor = layers.Dense(8, activation='relu')(encoded)
            predictor = layers.Dropout(0.2)(predictor)
            output = layers.Dense(1, activation='linear')(predictor)
            
        elif model_type == 'bet':
            encoded = layers.Dense(128, activation='relu')(input_layer)
            encoded = layers.BatchNormalization()(encoded)
            encoded = layers.Dropout(0.4)(encoded)
            encoded = layers.Dense(64, activation='relu')(encoded)
            encoded = layers.BatchNormalization()(encoded)
            encoded = layers.Dropout(0.3)(encoded)
            encoded = layers.Dense(32, activation='relu')(encoded)
            
            decoded = layers.Dense(64, activation='relu')(encoded)
            decoded = layers.BatchNormalization()(decoded)
            decoded = layers.Dropout(0.3)(decoded)
            decoded = layers.Dense(128, activation='relu')(decoded)
            decoded = layers.BatchNormalization()(decoded)
            decoded = layers.Dropout(0.4)(decoded)
            decoded = layers.Dense(input_shape, activation='linear')(decoded)
            
            predictor = layers.Dense(16, activation='relu')(encoded)
            predictor = layers.Dropout(0.3)(predictor)
            output = layers.Dense(1, activation='linear')(predictor)
            
        elif model_type == 'win':
            encoded = layers.Dense(256, activation='relu')(input_layer)
            encoded = layers.BatchNormalization()(encoded)
            encoded = layers.Dropout(0.4)(encoded)
            encoded = layers.Dense(128, activation='relu')(encoded)
            encoded = layers.BatchNormalization()(encoded)
            encoded = layers.Dropout(0.3)(encoded)
            encoded = layers.Dense(64, activation='relu')(encoded)
            
            decoded = layers.Dense(128, activation='relu')(encoded)
            decoded = layers.BatchNormalization()(decoded)
            decoded = layers.Dropout(0.3)(decoded)
            decoded = layers.Dense(256, activation='relu')(decoded)
            decoded = layers.BatchNormalization()(decoded)
            decoded = layers.Dropout(0.4)(decoded)
            decoded = layers.Dense(input_shape, activation='linear')(decoded)
            
            predictor = layers.Dense(32, activation='relu')(encoded)
            predictor = layers.Dropout(0.3)(predictor)
            predictor = layers.Dense(16, activation='relu')(predictor)
            predictor = layers.Dropout(0.2)(predictor)
            output = layers.Dense(1, activation='linear')(predictor)
        
        model = keras.Model(input_layer, [output, decoded])
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss=['mse', 'mse'],
            loss_weights=[0.7, 0.3],
            metrics={'dense_output': ['mae'], 'decoded_output': ['mae']}
        )
        
        return model
    
    def generate_random_data(self, X_train, n_samples=1000):
        """Generar datos aleatorios basados en estadísticas de entrenamiento"""
        np.random.seed(42)
        random_data = np.zeros((n_samples, X_train.shape[1]))
        
        for i in range(X_train.shape[1]):
            feature_mean = X_train[:, i].mean()
            feature_std = X_train[:, i].std()
            feature_min = X_train[:, i].min()
            feature_max = X_train[:, i].max()
            
            random_data[:, i] = np.random.normal(feature_mean, feature_std, size=n_samples)
            random_data[:, i] = np.clip(random_data[:, i], feature_min, feature_max)
        
        return random_data
    
    def create_individual_feature_visualizations(self, X_real_scaled, y_real, X_random_scaled, y_pred_random, 
                                           X_real_original, X_random_original, feature_names, model_name, 
                                           architecture, output_dir='visualizations', filter_outliers=True):
        """Crear visualizaciones individuales para cada variable con valores sin escalar"""
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        n_features = X_real_original.shape[1]
        print(f"\n=== CREANDO VISUALIZACIONES PARA {n_features} VARIABLES - {model_name.upper()} ({architecture}) ===")
        
        # FIX: Ensure all arrays have the same length
        min_len = min(len(X_real_original), len(y_real))
        X_real_original = X_real_original[:min_len]
        y_real = y_real[:min_len]
        
        # Also ensure random data matches
        min_random_len = min(len(X_random_original), len(y_pred_random))
        X_random_original = X_random_original[:min_random_len]
        y_pred_random = y_pred_random[:min_random_len]
        
        print(f"Real data points: {len(X_real_original)}, Random data points: {len(X_random_original)}")
        
        fig, axes = plt.subplots(2, (n_features + 1) // 2, figsize=(6 * ((n_features + 1) // 2), 12))
        if n_features == 1:
            axes = [axes]
        elif (n_features + 1) // 2 == 1:
            axes = axes.reshape(-1, 1)
        axes = axes.flatten()
        
        for i, feature_name in enumerate(feature_names):
            if i >= len(axes):
                break
                
            df_viz = pd.DataFrame({
                'Feature_Value': np.concatenate([X_real_original[:, i], X_random_original[:, i]]),
                'Predictions': np.concatenate([y_real, y_pred_random]),
                'Data_Type': ['Real'] * len(X_real_original) + ['Random'] * len(X_random_original)
            })
            
            if filter_outliers:
                feature_p5 = df_viz['Feature_Value'].quantile(0.05)
                feature_p95 = df_viz['Feature_Value'].quantile(0.95)
                pred_p5 = df_viz['Predictions'].quantile(0.05)
                pred_p95 = df_viz['Predictions'].quantile(0.95)
                
                mask = (
                    (df_viz['Feature_Value'] >= feature_p5) & 
                    (df_viz['Feature_Value'] <= feature_p95) &
                    (df_viz['Predictions'] >= pred_p5) &
                    (df_viz['Predictions'] <= pred_p95)
                )
                
                df_viz_filtered = df_viz[mask].copy()
                removed_points = len(df_viz) - len(df_viz_filtered)
                
                if removed_points > 0:
                    print(f"{feature_name}: Removidos {removed_points} outliers en visualización")
                
                df_viz = df_viz_filtered
            
            ax = axes[i]
            
            real_data = df_viz[df_viz['Data_Type'] == 'Real']
            random_data = df_viz[df_viz['Data_Type'] == 'Random']
            
            ax.scatter(real_data['Feature_Value'], real_data['Predictions'], 
                    color='blue', alpha=0.6, s=30, label='Real Data')
            ax.scatter(random_data['Feature_Value'], random_data['Predictions'], 
                    color='red', alpha=0.6, s=30, label='Random Data')
            
            colors = {'Real': 'blue', 'Random': 'red'}
            for data_type, color in colors.items():
                subset = df_viz[df_viz['Data_Type'] == data_type]
                if len(subset) > 1:
                    lr = LinearRegression()
                    X_trend = subset['Feature_Value'].values.reshape(-1, 1)
                    y_trend = subset['Predictions'].values
                    
                    if np.var(X_trend) > 1e-10:
                        lr.fit(X_trend, y_trend)
                        
                        x_line = np.linspace(subset['Feature_Value'].min(), 
                                        subset['Feature_Value'].max(), 100)
                        y_line = lr.predict(x_line.reshape(-1, 1))
                        
                        try:
                            r2 = lr.score(X_trend, y_trend)
                            ax.plot(x_line, y_line, color=color, linestyle='--', 
                                alpha=0.8, linewidth=2, 
                                label=f'{data_type} Trend (R²={r2:.3f})')
                        except:
                            pass
            
            ax.set_xlabel(f'{feature_name}')
            ax.set_ylabel('Predicciones')
            ax.set_title(f'{feature_name}\n({len(df_viz)} puntos) - Valores Originales')
            ax.legend(fontsize=8)
            ax.grid(True, alpha=0.3)
        
        for i in range(len(feature_names), len(axes)):
            axes[i].set_visible(False)
        
        filter_status = "CON FILTRO" if filter_outliers else "SIN FILTRO"
        fig.suptitle(f'Análisis por Variable - {model_name.upper()} ({architecture}) ({filter_status}) - Valores Originales', 
                    fontsize=16, y=0.98)
        plt.tight_layout()
        plt.subplots_adjust(top=0.93)
        
        suffix = "_filtered" if filter_outliers else "_original"
        filename = f"{output_dir}/{model_name}_{architecture.lower()}_individual_features{suffix}.png"
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"Gráfico guardado: {filename}")
        
    def train_and_visualize_all(self, df, architectures=['MLP'], filter_outliers=True, 
                               outlier_method='iqr', filter_viz_outliers=True, 
                               n_random_samples=1000, output_dir='multi_arch_results'):
        """Entrenar múltiples arquitecturas y crear visualizaciones"""
        print("=" * 80)
        print("ENTRENANDO MÚLTIPLES ARQUITECTURAS CON VISUALIZACIONES")
        print(f"Arquitecturas seleccionadas: {', '.join(architectures)}")
        print("=" * 80)
        
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        viz_dir = f"{output_dir}/visualizations"
        if not os.path.exists(viz_dir):
            os.makedirs(viz_dir)
        
        if filter_outliers:
            df_processed = self.filter_outliers(df, method=outlier_method)
        else:
            df_processed = df.copy()
            print(f"\nSin filtrado de outliers - Dataset: {len(df_processed)} registros")
        
        self.analyze_business_patterns(df_processed)
        
        X_base = df_processed[['INITIAL_AMOUNT','AVG_BET','Cluster','Weekday','Weekend','Month']]
        y_tiempo = df_processed['TIME_ON_DEVICE_MIN']
        y_bet = df_processed['BET_TOTAL'] 
        y_win = df_processed['WIN_TOTAL']
        
        X_base_train, X_base_test, y_tiempo_train, y_tiempo_test = train_test_split(
            X_base, y_tiempo, test_size=0.2, random_state=42, 
            stratify=pd.cut(y_tiempo, bins=5, labels=False)
        )
        
        train_idx, test_idx = X_base_train.index, X_base_test.index
        
        all_results = {'tiempo': {}, 'bet': {}, 'win': {}}
        all_visualizations = {'tiempo': {}, 'bet': {}, 'win': {}}
        
        print(f"\n{'='*50}")
        print("MODELOS PARA PREDICCIÓN DE TIEMPO")
        print(f"{'='*50}")
        
        X_tiempo_train, feature_names_tiempo = self.create_business_features(X_base_train)
        X_tiempo_test, _ = self.create_business_features(X_base_test)
        
        self.feature_names['tiempo'] = feature_names_tiempo
        
        scaler_tiempo = RobustScaler()
        X_tiempo_train_scaled = scaler_tiempo.fit_transform(X_tiempo_train)
        X_tiempo_test_scaled = scaler_tiempo.transform(X_tiempo_test)
        self.scalers['tiempo'] = scaler_tiempo
        
        best_tiempo_r2 = -np.inf
        best_tiempo_arch = None
        best_tiempo_pred_train = None
        best_tiempo_pred_test = None
        best_tiempo_model = None
        
        for arch in architectures:
            try:
                model, predictions, r2, mae, rmse, X_train_orig, X_test_orig = self.train_single_model(
                    X_tiempo_train_scaled, X_tiempo_test_scaled, 
                    y_tiempo_train.values, y_tiempo_test.values,
                    'tiempo', 'tiempo', arch
                )
                
                if model is not None:
                    all_results['tiempo'][arch] = {
                        'R2': r2, 'MAE': mae, 'RMSE': rmse, 
                        'model': model, 'predictions': predictions
                    }
                    
                    X_random_tiempo = self.generate_random_data(X_tiempo_train_scaled, n_random_samples)
                    
                    X_tiempo_test_original = scaler_tiempo.inverse_transform(X_test_orig)
                    X_random_tiempo_original = scaler_tiempo.inverse_transform(X_random_tiempo)
                    
                    if arch in ['RNN', 'LSTM', 'GRU', 'CNN1D', 'TRANSFORMER']:
                        X_random_prepared = self.prepare_sequence_data(X_random_tiempo, min(10, len(X_tiempo_train_scaled)))
                    elif arch == 'CNN2D':
                        X_random_prepared = self.prepare_cnn2d_data(X_random_tiempo)
                    else:
                        X_random_prepared = X_random_tiempo
                    
                    if arch == 'AUTOENCODER':
                        y_random_tiempo_pred = model.predict(X_random_prepared, verbose=0)[0].flatten()
                    else:
                        y_random_tiempo_pred = model.predict(X_random_prepared, verbose=0).flatten()
                    
                    viz = self.create_individual_feature_visualizations(
                        X_tiempo_test_scaled, y_tiempo_test.values, 
                        X_random_tiempo, y_random_tiempo_pred, 
                        X_tiempo_test_original, X_random_tiempo_original,
                        feature_names_tiempo, 'tiempo', arch, viz_dir, filter_viz_outliers
                    )
                    all_visualizations['tiempo'][arch] = viz
                    
                    if r2 > best_tiempo_r2:
                        best_tiempo_r2 = r2
                        best_tiempo_arch = arch
                        best_tiempo_model = model
                        # Generate predictions for FULL train/test sets
                        if arch in ['RNN', 'LSTM', 'GRU', 'CNN1D', 'TRANSFORMER']:
                            X_train_for_pred = self.prepare_sequence_data(X_tiempo_train_scaled, min(10, len(X_tiempo_train_scaled)))
                            X_test_for_pred = self.prepare_sequence_data(X_tiempo_test_scaled, min(10, len(X_tiempo_test_scaled)))
                        elif arch == 'CNN2D':
                            X_train_for_pred = self.prepare_cnn2d_data(X_tiempo_train_scaled)
                            X_test_for_pred = self.prepare_cnn2d_data(X_tiempo_test_scaled)
                        else:
                            X_train_for_pred = X_tiempo_train_scaled
                            X_test_for_pred = X_tiempo_test_scaled
                        
                        if arch == 'AUTOENCODER':
                            best_tiempo_pred_train = model.predict(X_train_for_pred, verbose=0)[0].flatten()
                            best_tiempo_pred_test = model.predict(X_test_for_pred, verbose=0)[0].flatten()
                        else:
                            best_tiempo_pred_train = model.predict(X_train_for_pred, verbose=0).flatten()
                            best_tiempo_pred_test = model.predict(X_test_for_pred, verbose=0).flatten()
                        
            except Exception as e:
                print(f"Error con {arch} para tiempo: {e}")
                import traceback
                traceback.print_exc()
                continue
        
        print(f"\n🏆 Mejor modelo TIEMPO: {best_tiempo_arch} (R² = {best_tiempo_r2:.4f})")
        
        # FIX: Ensure predictions match the original DataFrame lengths
        print(f"\nAjustando predicciones de TIEMPO:")
        print(f"  Train original: {len(X_base_train)}, Predicciones: {len(best_tiempo_pred_train)}")
        print(f"  Test original: {len(X_base_test)}, Predicciones: {len(best_tiempo_pred_test)}")
        
        print(f"\n{'='*50}")
        print("MODELOS PARA PREDICCIÓN DE BET TOTAL")
        print(f"{'='*50}")
        
        X_bet_train, feature_names_bet = self.create_business_features(X_base_train, best_tiempo_pred_train)
        X_bet_test, _ = self.create_business_features(X_base_test, best_tiempo_pred_test)
        
        self.feature_names['bet'] = feature_names_bet
        
        scaler_bet = RobustScaler()
        X_bet_train_scaled = scaler_bet.fit_transform(X_bet_train)
        X_bet_test_scaled = scaler_bet.transform(X_bet_test)
        self.scalers['bet'] = scaler_bet
        
        best_bet_r2 = -np.inf
        best_bet_arch = None
        best_bet_pred_test = None
        best_bet_model = None
        
        for arch in architectures:
            try:
                model, predictions, r2, mae, rmse, X_train_orig, X_test_orig = self.train_single_model(
                    X_bet_train_scaled, X_bet_test_scaled,
                    y_bet.loc[train_idx].values, y_bet.loc[test_idx].values,
                    'bet', 'bet_total', arch
                )
                
                if model is not None:
                    all_results['bet'][arch] = {
                        'R2': r2, 'MAE': mae, 'RMSE': rmse, 
                        'model': model, 'predictions': predictions
                    }
                    
                    X_random_bet = self.generate_random_data(X_bet_train_scaled, n_random_samples)
                    
                    X_bet_test_original = scaler_bet.inverse_transform(X_test_orig)
                    X_random_bet_original = scaler_bet.inverse_transform(X_random_bet)
                    
                    if arch in ['RNN', 'LSTM', 'GRU', 'CNN1D', 'TRANSFORMER']:
                        X_random_prepared = self.prepare_sequence_data(X_random_bet, min(10, len(X_bet_train_scaled)))
                    elif arch == 'CNN2D':
                        X_random_prepared = self.prepare_cnn2d_data(X_random_bet)
                    else:
                        X_random_prepared = X_random_bet
                    
                    if arch == 'AUTOENCODER':
                        y_random_bet_pred = model.predict(X_random_prepared, verbose=0)[0].flatten()
                    else:
                        y_random_bet_pred = model.predict(X_random_prepared, verbose=0).flatten()
                    
                    viz = self.create_individual_feature_visualizations(
                        X_bet_test_scaled, y_bet.loc[test_idx].values,
                        X_random_bet, y_random_bet_pred,
                        X_bet_test_original, X_random_bet_original,
                        feature_names_bet, 'bet', arch, viz_dir, filter_viz_outliers
                    )
                    all_visualizations['bet'][arch] = viz
                    
                    if r2 > best_bet_r2:
                        best_bet_r2 = r2
                        best_bet_arch = arch
                        best_bet_model = model
                        
                        if arch in ['RNN', 'LSTM', 'GRU', 'CNN1D', 'TRANSFORMER']:
                            X_test_for_pred = self.prepare_sequence_data(X_bet_test_scaled, min(10, len(X_bet_test_scaled)))
                        elif arch == 'CNN2D':
                            X_test_for_pred = self.prepare_cnn2d_data(X_bet_test_scaled)
                        else:
                            X_test_for_pred = X_bet_test_scaled
                        
                        if arch == 'AUTOENCODER':
                            best_bet_pred_test = model.predict(X_test_for_pred, verbose=0)[0].flatten()
                        else:
                            best_bet_pred_test = model.predict(X_test_for_pred, verbose=0).flatten()
                        
            except Exception as e:
                print(f"Error con {arch} para bet: {e}")
                import traceback
                traceback.print_exc()
                continue
        
        print(f"\n🏆 Mejor modelo BET: {best_bet_arch} (R² = {best_bet_r2:.4f})")
        
        print(f"\n{'='*50}")
        print("MODELOS PARA PREDICCIÓN DE WIN TOTAL")
        print(f"{'='*50}")
        
        # Generate bet predictions for train set
        if best_bet_arch in ['RNN', 'LSTM', 'GRU', 'CNN1D', 'TRANSFORMER']:
            X_bet_train_prep = self.prepare_sequence_data(X_bet_train_scaled, min(10, len(X_bet_train_scaled)))
        elif best_bet_arch == 'CNN2D':
            X_bet_train_prep = self.prepare_cnn2d_data(X_bet_train_scaled)
        else:
            X_bet_train_prep = X_bet_train_scaled
        
        if best_bet_arch == 'AUTOENCODER':
            best_bet_pred_train = best_bet_model.predict(X_bet_train_prep, verbose=0)[0].flatten()
        else:
            best_bet_pred_train = best_bet_model.predict(X_bet_train_prep, verbose=0).flatten()
        
        print(f"\nAjustando predicciones de BET:")
        print(f"  Train original: {len(X_base_train)}, Predicciones: {len(best_bet_pred_train)}")
        print(f"  Test original: {len(X_base_test)}, Predicciones: {len(best_bet_pred_test)}")
        
        X_win_train, feature_names_win = self.create_business_features(
            X_base_train, best_tiempo_pred_train, best_bet_pred_train
        )
        X_win_test, _ = self.create_business_features(
            X_base_test, best_tiempo_pred_test, best_bet_pred_test
        )
        
        self.feature_names['win'] = feature_names_win
        
        scaler_win = RobustScaler()
        X_win_train_scaled = scaler_win.fit_transform(X_win_train)
        X_win_test_scaled = scaler_win.transform(X_win_test)
        self.scalers['win'] = scaler_win
        
        best_win_r2 = -np.inf
        best_win_arch = None
        
        for arch in architectures:
            try:
                model, predictions, r2, mae, rmse, X_train_orig, X_test_orig = self.train_single_model(
                    X_win_train_scaled, X_win_test_scaled,
                    y_win.loc[train_idx].values, y_win.loc[test_idx].values,
                    'win', 'win_total', arch
                )
                
                if model is not None:
                    all_results['win'][arch] = {
                        'R2': r2, 'MAE': mae, 'RMSE': rmse, 
                        'model': model, 'predictions': predictions
                    }
                    
                    X_random_win = self.generate_random_data(X_win_train_scaled, n_random_samples)
                    
                    X_win_test_original = scaler_win.inverse_transform(X_test_orig)
                    X_random_win_original = scaler_win.inverse_transform(X_random_win)
                    
                    if arch in ['RNN', 'LSTM', 'GRU', 'CNN1D', 'TRANSFORMER']:
                        X_random_prepared = self.prepare_sequence_data(X_random_win, min(10, len(X_win_train_scaled)))
                    elif arch == 'CNN2D':
                        X_random_prepared = self.prepare_cnn2d_data(X_random_win)
                    else:
                        X_random_prepared = X_random_win
                    
                    if arch == 'AUTOENCODER':
                        y_random_win_pred = model.predict(X_random_prepared, verbose=0)[0].flatten()
                    else:
                        y_random_win_pred = model.predict(X_random_prepared, verbose=0).flatten()
                    
                    viz = self.create_individual_feature_visualizations(
                        X_win_test_scaled, y_win.loc[test_idx].values,
                        X_random_win, y_random_win_pred,
                        X_win_test_original, X_random_win_original,
                        feature_names_win, 'win', arch, viz_dir, filter_viz_outliers
                    )
                    all_visualizations['win'][arch] = viz
                    
                    if r2 > best_win_r2:
                        best_win_r2 = r2
                        best_win_arch = arch
                        
            except Exception as e:
                print(f"Error con {arch} para win: {e}")
                import traceback
                traceback.print_exc()
                continue
        
        if best_win_arch:
            print(f"\n🏆 Mejor modelo WIN: {best_win_arch} (R² = {best_win_r2:.4f})")
        
        print(f"\n{'='*80}")
        print("RESUMEN COMPARATIVO DE TODAS LAS ARQUITECTURAS")
        print(f"{'='*80}")
        
        comparison_data = []
        for target in ['tiempo', 'bet', 'win']:
            for arch in architectures:
                if arch in all_results[target]:
                    result = all_results[target][arch]
                    comparison_data.append({
                        'Target': target.upper(),
                        'Architecture': arch,
                        'R2_Score': result['R2'],
                        'MAE': result['MAE'],
                        'RMSE': result['RMSE'],
                        'N_Features': len(self.feature_names[target]),
                        'Features': ', '.join(self.feature_names[target])
                    })
        
        comparison_df = pd.DataFrame(comparison_data)
        comparison_filename = f"{output_dir}/architecture_comparison.csv"
        comparison_df.to_csv(comparison_filename, index=False)
        print(f"✓ Comparación guardada: {comparison_filename}")
        
        print(f"\n🏆 MEJORES MODELOS POR OBJETIVO:")
        print(f"   TIEMPO: {best_tiempo_arch} (R² = {best_tiempo_r2:.4f})")
        print(f"   BET: {best_bet_arch} (R² = {best_bet_r2:.4f})")
        if best_win_arch:
            print(f"   WIN: {best_win_arch} (R² = {best_win_r2:.4f})")
        
        arch_rankings = {}
        for arch in architectures:
            r2_scores = []
            for target in ['tiempo', 'bet', 'win']:
                if arch in all_results[target]:
                    r2_scores.append(all_results[target][arch]['R2'])
            if r2_scores:
                arch_rankings[arch] = np.mean(r2_scores)
        
        print(f"\n📊 RANKING DE ARQUITECTURAS (R² promedio):")
        for i, (arch, avg_r2) in enumerate(sorted(arch_rankings.items(), key=lambda x: x[1], reverse=True), 1):
            print(f"   {i}. {arch}: {avg_r2:.4f}")
        
        print(f"\n📁 Resultados guardados en: '{output_dir}'")
        print(f"📁 Visualizaciones guardadas en: '{viz_dir}'")
        
        return {
            'results': all_results,
            'comparison': comparison_df,
            'visualizations': all_visualizations,
            'best_models': {
                'tiempo': best_tiempo_arch,
                'bet': best_bet_arch,
                'win': best_win_arch
            }
        }
    def train_single_model(self, X_train, X_test, y_train, y_test, model_type, target_name, architecture='MLP'):
        """Entrenar un solo modelo con arquitectura específica"""
        print(f"\n=== {architecture} - {target_name.upper()} ===")
        
        X_train_original = X_train.copy()
        X_test_original = X_test.copy()
        
        if architecture in ['RNN', 'LSTM', 'GRU', 'CNN1D', 'TRANSFORMER']:
            sequence_length = min(10, len(X_train))
            X_train_prepared = self.prepare_sequence_data(X_train, sequence_length)
            X_test_prepared = self.prepare_sequence_data(X_test, sequence_length)
            
            if len(y_train) > len(X_train_prepared):
                y_train = y_train[len(y_train) - len(X_train_prepared):]
            if len(y_test) > len(X_test_prepared):
                y_test = y_test[len(y_test) - len(X_test_prepared):]
                
            input_shape = X_train_prepared.shape[1:]
            
        elif architecture == 'CNN2D':
            X_train_prepared = self.prepare_cnn2d_data(X_train)
            X_test_prepared = self.prepare_cnn2d_data(X_test)
            input_shape = X_train_prepared.shape[1:]
            
        else:
            X_train_prepared = X_train_original
            X_test_prepared = X_test_original
            input_shape = X_train.shape[1]
        
        model = None
        if architecture == 'MLP':
            model = self.create_mlp_model(input_shape, model_type)
        elif architecture == 'RNN':
            model = self.create_rnn_model(input_shape, model_type)
        elif architecture == 'LSTM':
            model = self.create_lstm_model(input_shape, model_type)
        elif architecture == 'GRU':
            model = self.create_gru_model(input_shape, model_type)
        elif architecture == 'CNN1D':
            model = self.create_cnn1d_model(input_shape, model_type)
        elif architecture == 'CNN2D':
            model = self.create_cnn2d_model(input_shape, model_type)
        elif architecture == 'TRANSFORMER':
            model = self.create_transformer_model(input_shape, model_type)
        elif architecture == 'AUTOENCODER':
            model = self.create_autoencoder_model(input_shape, model_type)
        else:
            print(f"Error: Arquitectura '{architecture}' no reconocida")
    
    def analyze_business_patterns(self, df):
        """Análisis de patrones de negocio"""
        print(f"\n=== ANÁLISIS DE PATRONES DE NEGOCIO ===")
        reinvested = df['BET_TOTAL'] > df['INITIAL_AMOUNT']
        print(f"Sesiones con reinversión: {reinvested.sum()} ({reinvested.sum()/len(df)*100:.1f}%)")
        
        net_winners = df['FINAL_AMOUNT'] > df['INITIAL_AMOUNT']
        net_losers = df['FINAL_AMOUNT'] < df['INITIAL_AMOUNT']
        
        print(f"Ganadores netos: {net_winners.sum()} ({net_winners.sum()/len(df)*100:.1f}%)")
        print(f"Perdedores netos: {net_losers.sum()} ({net_losers.sum()/len(df)*100:.1f}%)")
        
        win_bet_corr = np.corrcoef(df['WIN_TOTAL'], df['BET_TOTAL'])[0,1]
        print(f"Correlación WIN vs BET: {win_bet_corr:.3f}")
        
        print(f"Dataset shape: {df.shape}")
        print(f"Rango TIEMPO: {df['TIME_ON_DEVICE_MIN'].min():.1f} - {df['TIME_ON_DEVICE_MIN'].max():.1f}")
        print(f"Rango BET: {df['BET_TOTAL'].min():.1f} - {df['BET_TOTAL'].max():.1f}")
        print(f"Rango WIN: {df['WIN_TOTAL'].min():.1f} - {df['WIN_TOTAL'].max():.1f}")  

In [12]:
casino_ml = MultiArchitectureCasinoML()

results = casino_ml.train_and_visualize_all(
        df_data_general,
        architectures=['RNN'],
        filter_viz_outliers=False,
        filter_outliers=False,
        n_random_samples=1000,
        output_dir='mlp_only_results'
    )

ENTRENANDO MÚLTIPLES ARQUITECTURAS CON VISUALIZACIONES
Arquitecturas seleccionadas: RNN

Sin filtrado de outliers - Dataset: 209079 registros

=== ANÁLISIS DE PATRONES DE NEGOCIO ===
Sesiones con reinversión: 160049 (76.5%)
Ganadores netos: 36184 (17.3%)
Perdedores netos: 172469 (82.5%)
Correlación WIN vs BET: 0.870
Dataset shape: (209079, 41)
Rango TIEMPO: 0.1 - 537.9
Rango BET: 0.0 - 134666.6
Rango WIN: 0.0 - 461502.0

MODELOS PARA PREDICCIÓN DE TIEMPO

=== RNN - TIEMPO ===
Error con RNN para tiempo: cannot unpack non-iterable NoneType object

🏆 Mejor modelo TIEMPO: None (R² = -inf)

Ajustando predicciones de TIEMPO:


Traceback (most recent call last):
  File "/var/folders/ld/2yb_9mvs5t90r3x4r3v3btbc0000gn/T/ipykernel_33721/1356875699.py", line 749, in train_and_visualize_all
    model, predictions, r2, mae, rmse, X_train_orig, X_test_orig = self.train_single_model(
TypeError: cannot unpack non-iterable NoneType object


TypeError: object of type 'NoneType' has no len()

In [ ]:
--------------------------------------------------------------------------- ValueError Traceback (most recent call last) Cell In[8], line 3 1 casino_ml = MultiArchitectureCasinoML() ----> 3 results = casino_ml.train_and_visualize_all( 4 df_data_general, 5 architectures=['RNN'], 6 filter_viz_outliers=False, 7 filter_outliers=False, 8 n_random_samples=1000, 9 output_dir='mlp_only_results' 10 ) Cell In[4], line 922 919 print("MODELOS PARA PREDICCIÓN DE BET TOTAL") 920 print(f"{'='*50}") --> 922 X_bet_train, feature_names_bet = self.create_business_features(X_base_train, best_tiempo_pred_train) 923 X_bet_test, _ = self.create_business_features(X_base_test, best_tiempo_pred_test) 925 self.feature_names['bet'] = feature_names_bet Cell In[4], line 85 82 feature_names = ['INITIAL_AMOUNT', 'AVG_BET', 'Cluster'] 84 if tiempo_pred is not None: ---> 85 features['tiempo_pred'] = tiempo_pred 86 feature_names.append('tiempo_pred')
...
 560 "does not match length of index " 561 f"({len(index)})" 562 ) ValueError: Length of values (167254) does not match length of index (167263)